<a href="https://colab.research.google.com/github/ghoshatanu857/Instrument_Automation/blob/main/ChargeLaserDynamics/modifying_Charge_Laser_dynamics_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Modeling Charge State Dynamics to explain the preference of 975nm over 1064nm as trapped Laser
*Atanu Ghosh* (PH20D204), Department of Physics, IITM

### Preliminary  <span style= 'color: Green'> $\textbf{Imports}$ </span>

In [ ]:
import numpy as np
import pandas as pd
import html, array, sympy, sys, time, sympy, os, glob, joblib, math, shutil
from statistics import mean
from scipy.optimize import fsolve
from scipy.interpolate import interp1d
from sympy import symbols, Eq, solve
from scipy import signal
from plotly import graph_objs as go
from scipy import stats
import plotly.io as pio
import plotly.express as px
from tqdm.notebook import trange
from plotly.subplots import make_subplots
from joblib import Parallel, delayed
from importlib import reload
from scipy.optimize import curve_fit
from scipy.stats import chisquare
import my_func as fun
reload(fun)

In [ ]:
# Current Directory
path = os.getcwd()
path
# from google.colab import drive
# drive.mount('/content/drive')
# path = '/content/drive/MyDrive'

In [ ]:
fig_template = go.layout.Template()
fig_template.layout = {
    'template': 'simple_white+presentation',
    'autosize': False,
    'width': 800,
    'height': 600,
    # 'opacity': 0.2,
    'xaxis': {
        'ticks': 'inside',
        'mirror': 'ticks',
        'linewidth': 1.5+0.5,
        'tickwidth': 1.5+0.5,
        'ticklen': 6,
        'showline': True,
        'showgrid': False,
        'zerolinecolor': 'white',
        },
    'yaxis': {
        'ticks': 'inside',
        'mirror': 'ticks',
        'linewidth': 1.5+0.5,
        'tickwidth': 1.5+0.5,
        'ticklen': 6,
        'showline': True,
        'showgrid': False,
        'zerolinecolor': 'white'
        },
    'font':{'family':'mathjax',
            'size': 22,
            }
}

In [ ]:
alphabet=px.colors.qualitative.Alphabet
plotly=px.colors.qualitative.Plotly
g10=px.colors.qualitative.G10
vivid=px.colors.qualitative.Vivid
pastel=px.colors.qualitative.Pastel

### All the <span style= 'color: Green'> $\textbf{Functions}$ </span>
> Imported from the seperate python file "my_func.py"

In [ ]:
# def laser_scaling(data,laser_power,window):
#     x=data[:,0]
#     lowIndex = np.where(x.astype(int) == window[0])[0][0]
#     highIndex = np.where(x.astype(int) == window[1])[0][0]
#     scale=[]
#     for i in range(len(laser_power)):
#         y = data[:,i+1][lowIndex:highIndex]
#         scale.append(np.sum(y)/laser_power[i])
#     return np.array(scale)

In [ ]:
# #function to correct the dip
# def correction(x,y,window):
#     # x,y,window = args  #*args passing to function
#     lowIndex = np.where(x.astype(int) == window[0])[0][0]
#     highIndex = np.where(x.astype(int) == window[1])[0][0]
#     y1=[];y2=[];x1=[];x2=[]
#     points = 1            #points beyond the window to interpolet
#     for i in range(points):
#         y1.append(y[lowIndex-i]) ; y2.append(y[highIndex+i])
#         x1.append(x[lowIndex-i]) ; x2.append(x[highIndex+i])
#     y1 = np.sort(np.array(y1)); y2 = np.array(y2);x1 = np.sort(np.array(x1)); x2 = np.array(x2)
#     yold = np.concatenate((y1,y2)); xold = np.concatenate((x1,x2)) ; xnew = x[lowIndex:highIndex+1]
#     func = interp1d(xold,yold)
#     ynew = func(xnew)
#     # ynew = ynew + np.random.uniform(low=10.0, high=130.0, size=ynew.shape[0])
#     index = np.arange(lowIndex,highIndex+1,1)
#     yNew = np.insert(np.delete(y,index),index[0],ynew)
#     # yNew = yNew+uplift
#     return yNew

In [ ]:
# #Function for scaling
# def scaled(y):
#     scale  = np.max(y)-np.min(y)
#     y = (y - np.min(y))/scale
#     return y

In [ ]:
# def interpolation(x,y,xnew):
#     func=interp1d(x,y)
#     return func(xnew)

In [ ]:
# #### Function to get Integrated PL and Charge State Ratio
# def exp(x,y,limit,pl_range):
#     index = np.where(x.astype(int) == limit)[0][0]  # taking the index of the value
#     if type(pl_range)==np.ndarray:
#         index1 = np.where(x.astype(int) == pl_range[0])[0][0]  # taking the index of the value
#         index2 = np.where(x.astype(int) == pl_range[1])[0][0]  # taking the index of the value
#         x1 = x[index1:index]
#         x2 = x[index:index2]
#         y1 = y[index1:index]
#         y2 = y[index:index2]
#         # intensity = np.trapz(y,x)
#         # ratio = np.trapz(y2,x2)/np.trapz(y1,x1)
#         intensity = np.sum(y)
#         ratio = np.sum(y2)/np.sum(y1)
#         return np.array([intensity,ratio])
#     else:
#         y1 = y[:index]; y2 = y[index:]
#         intensity = np.sum(y); ratio = np.sum(y2)/np.sum(y1)
#         return np.array([intensity,ratio])

In [ ]:
# #function to extract data
# def extract_data(file_list,window):
#     rows = np.loadtxt(f'{file_list[0]}').shape[0]
#     # cols = len(file_list)+1
#     cols = len(file_list)     # to ignore the first not-good data
#     data_pl = np.zeros((rows,cols))
#     data_pl[:,0] = np.loadtxt(f'{file_list[0]}')[:,0]
#     # for i in range(len(file_list)):
#     for i in range(len(file_list)-1):
#         if type(window) == np.ndarray:
#             # data_pl[:,i+1] = correction(data_pl[:,0],np.loadtxt(f'{file_list[i]}')[:,1],window)
#             data_pl[:,i+1] = correction(data_pl[:,0],np.loadtxt(f'{file_list[i+1]}')[:,1],window)     # to ignore the first not-good data
#         else:
#             # data_pl[:,i+1] = np.loadtxt(f'{file_list[i]}')[:,1]
#             data_pl[:,i+1] = np.loadtxt(f'{file_list[i+1]}')[:,1]       # to ignore the first not-good data
#     return data_pl

In [ ]:
# #to get Chi_squared
# def chi_sqr(simul,exper):
#     diff = np.sum(np.square(simul-exper)/exper)
#     return diff

In [ ]:
# #To organize the data suitable for calculating chi_squared
# def organize(data,organize_index,dimension):
#     data_org = []
#     if dimension == 2:
#         if organize_index == 'i':
#             for i in range(data.shape[0]):
#                 data_org = data_org + data[i,:].tolist()
#         if organize_index == 'j':
#             for j in range(data.shape[1]):
#                 data_org = data_org + data[:,j].tolist()
#     # i=0;j=0
#     # if dimension == 3:
#     #     if organize_index == 'i':
#     #         for j in range(data.shape[0]):
#     #             for k in range(data.shape[1]):
#     #             data_org = data_org + data[i,:].tolist()
#     return np.array(data_org)

In [ ]:
# #function to get the Integrated PL and ratio for different IR and green
# def intensity(data,powers,cutOff,pl_range,dim):
#     if dim==3:
#         intensity= np.ones([data.shape[0],powers.shape[0]])
#         ratio= np.ones([data.shape[0],powers.shape[0]])
#         for i in range(data.shape[0]):
#             for j in range(powers.shape[0]):
#                 intensity[i,j] = exp(data[i,:,0],data[i,:,j+1],cutOff,pl_range)[0]
#                 ratio[i,j] = exp(data[i,:,0],data[i,:,j+1],cutOff,pl_range)[1]
#         return intensity,ratio
#     elif dim==2:
#         intensity= np.ones([powers.shape[0]])
#         ratio= np.ones([powers.shape[0]])
#         for i in range(powers.shape[0]):
#                 intensity[i] = exp(data[:,0],data[:,i+1],cutOff,pl_range)[0]
#                 ratio[i] = exp(data[:,0],data[:,i+1],cutOff,pl_range)[1]
#         return intensity,ratio

In [ ]:
# #### function to get the steady state population for green and IRs
# def green(y,*power):
#     power_green,power_975,power_1064 = power
#     nr1,nr1_p,nr2,nr2_p,nalpha1,nalpha2,nk1,nk2,nbeta1,nbeta2,nl1,nl2,ngamma1,ngamma2,ndelta = cof(power_green,power_975,power_1064)
#     evolve = np.array(([[(-(nr1*power_green)-(nk1*power_1064*power_1064)-(nl1*power_975*power_975)-(ndelta*power_975*power_975)),nr1_p,0,(nalpha2*power_green+nbeta2*power_1064+ngamma2*power_975),0],[(nr1*power_green+(nk1*power_1064*power_1064)+(nl1*power_975*power_975)),(-nalpha1*power_green-nr1_p-nbeta1*power_1064-ngamma1*power_975),0,0,0],[ndelta*power_975*power_975,(nalpha1*power_green+nbeta1*power_1064+ngamma1*power_975),(-nr2*power_green-(nk2*power_1064*power_1064)-(nl2*power_975*power_975)),nr2_p,0],[0,0,(nr2*power_green+(nk2*power_1064*power_1064)+(nl2*power_975*power_975)),(-nalpha2*power_green-nr2_p-nbeta2*power_1064-ngamma2*power_975),0],[1,1,1,1,-1]]))
#     state_green = np.matmul(evolve,np.abs([y[0],y[1],y[2],y[3],1]))
#     return state_green

In [ ]:
# def cof(power_green,power_975,power_1064):
#     #for green only
#     if power_975==0 and power_1064==0:
#         r1_ratio = 0.02
#         nr1_p=100; nr1=r1_ratio*nr1_p;  nr2_p=100; nr2=r1_ratio*nr2_p; nalpha1=nr1/10; nalpha2=nalpha1*4; nk1=nk2=nbeta1=nbeta2=nl1=nl2=ngamma1=ngamma2=ndelta=0

#     #for (green + 975)
#     if power_975!=0 and power_1064==0:
#         nr1_p=100; nr1=0.1*nr1_p;  nr2_p=100; nr2=0.1*nr2_p; nalpha2=3 ; nalpha1=1; nk1=nk2=nbeta1=nbeta2=0 ; nl1=nr1/10000;nl2=nr2/10000;ngamma1=nalpha1/3;ngamma2=nalpha2/3;ndelta=nr1/10000  #with delta
#         # nr1_p=100; nr1=0.1*nr1_p;  nr2_p=100; nr2=0.1*nr2_p; nalpha2=3 ; nalpha1=1; nk1=nk2=nbeta1=nbeta2=0 ; nl1=nr1/10000;nl2=nr2/10000;ngamma1=nalpha1/3;ngamma2=nalpha2/3;ndelta=0      #without delta

#     #for (green + 1064)
#     if power_975==0 and power_1064!=0:
#         nr1_p=100; nr1=0.1*nr1_p;  nr2_p=100; nr2=0.1*nr2_p; nalpha2=3 ; nalpha1=1; nl1=nl2=ngamma1=ngamma2=ndelta=0 ; nk1=nr1/10000;nk2=nr2/10000;nbeta1 = nalpha1;nbeta2 = nalpha2

#     co = np.array([nr1,nr1_p,nr2,nr2_p,nalpha1,nalpha2,nk1,nk2,nbeta1,nbeta2,nl1,nl2,ngamma1,ngamma2,ndelta])
#     return co

In [ ]:
# #function to scale the different dataSets with green powers
# def green_scale(data1,data2,power1,power2):
#     _,index1,index2=np.intersect1d(power1, power2, return_indices=True)
#     data1=data1[index1];data2=data2[index2]
#     return np.average(data1/data2)

In [ ]:
# #front background correction
# def background(data,upto):
#     average=np.mean(data[:,1:][:upto,:],axis=0)
#     data[:,1:]=data[:,1:]-average
#     return data

In [ ]:
x=np.array([3.77,3.104,3.051,3.014,2.98,2.691,2.356,2.193,1.932,1.701])
y=np.array([272,105,96.8,81,69,53,30,21,11.39,7.5])
xnew=np.array([10,30,60,80])

In [ ]:
func=interp1d(y,x)
ynew=func(xnew)

In [ ]:
ynew

In [ ]:
fig = go.Figure()
fig.add_scatter(x=y,y=x)
fig.update_layout(xaxis = dict(title = 'Wavelenght (nm)'),yaxis = dict(title = 'PL of 532nm (Second Set)'))
fig

### <span style= 'color: Green'> $\textbf{Experimental}$ </span> DataSets

In [ ]:
pl_range = np.array([555,785])
cutOff = 650  # cut_off wavelength to get the ratio

#### Extracting DataSets with Corrections

##### <span style='color: Red'> Calibration of <span style='color: Blue'>532nm</span> laser</span> for first dataSets

| Current (in $A$) | Power at sample plane (in $\mu W$) |
| :-: | :-: |
| 1.65 | 20 |
| 1.7 | 22 |
| 1.75 | 26.7 |
| 1.8 | 31 |
| 1.85 | 34 |
| 1.9 | 36 |
| 1.95 | 40 |
| 2 | 43 |
| 2.05 | 45.8 |
| 2.1 | 47|

In [ ]:
#using calibration for interpolation
current = np.array([1.65,1.7 , 1.75, 1.8 , 1.85, 1.9 , 1.95, 2.  , 2.05, 2.1 ])
power = np.array([20,22,26.7,31,34,36,40,43,45.8,47])
irs = ['975','1064']
window_975 = np.array([645,653])  # Dip_correcton window
window_1064 = np.array([697,705])
window_green = np.array([697,705])
# window_975 = 'False'
# window_1064 = 'False'
# window_green = 'False'

In [ ]:
ir_powers=[]
for files in os.listdir(path+"\\Exp_Data\\975nm"):
    file_list = files[len(f"{'975'}_"):-2]
    if len(file_list)<=5:
        ir_powers.append(int(file_list))
ir_powers=np.sort(np.array(ir_powers))

current_data=[]
for files in os.listdir(path+f"\\Exp_Data\\{irs[0]}nm\\{irs[0]}_{str(ir_powers[0])}mW"):
    file_list = files[:-4]
    if len(file_list)<=5:
        current_data.append(float(file_list))
current_data= np.sort(np.array(current_data[1:]))  #starting from index 1 to ignore the first data
green_powers= fun.interpolation(current,power,current_data)
current_data

greenOnly_powers = green_powers[0::2]

###### First DataSets

In [ ]:
for i in irs:
    globals()['data_'+f'{i}']=[]
    for j in ir_powers:
        txt_files = glob.glob(path+f"\\Exp_Data\\{i}nm\\{i}_{j}mW\\*.txt")
        globals()['data_'+f'{i}_'+f'{j}'] = fun.background(fun.extract_data(txt_files,window=globals()['window_'+f'{i}']),upto=50)  #like : data_1064_16    with corrections
        globals()['data_'+f'{i}'].append(globals()['data_'+f'{i}_'+f'{j}'])   #like data_1064
    globals()['data_'+f'{i}']=np.array(globals()['data_'+f'{i}'])

# data_975[0,:,6] = data_975[0,:,6]+200   #correction for over_correcting background noise

#only green dataSets
txt_files = glob.glob(path+"\\Exp_Data\\green\\*.txt")
data_green = fun.background(fun.extract_data(txt_files,window=window_green),upto=50)

In [ ]:
intensity_1064,ratio_1064 = fun.intensity(data_1064,green_powers,cutOff,pl_range=pl_range,dim=3)
intensity_975,ratio_975 = fun.intensity(data_975,green_powers,cutOff,pl_range=0,dim=3)
intensity_green,ratio_green = fun.intensity(data_green,greenOnly_powers,cutOff,pl_range=pl_range,dim=2)

##### <span style='color: Red'> Calibration of <span style='color: Blue'>532nm</span> laser</span> for second dataSets

| Current (in $A$) | Power at sample plane (in $\mu W$) |
| :-: | :-: |
| 2.3 | 68 |
| 2.18 | 84 |
| 2.1 | 82.7 |
| 2.011 | 66 |
| 1.8 | 46 |
| 1.521 | 36 |
| 1.4 | 27 |
| 1.2 | 24 |
| 1.1 | 19 |
| 1 | 22|
| 0.8 | 17|
| 0.627 | 8.5|

In [ ]:
#refernces
irs = ['975','1064']
scale_window = np.array([525,535])  #window to get the scaling of intensity with laser_power
current_new = np.array([2.3,2.18,2.1,2.011,1.8,1.521,1.4,1.2,1.1,1,0.8,0.627])
power_new = np.array([68,84,82.7,66,46,36,27,24,19,22,17,8.5])
window_975 = np.array([690,702])  # Dip_correcton window
window_1064 = np.array([698,710])
window_green = np.array([694,703])
# window_975 = 'False'
# window_1064 = 'False'
# window_green = 'False'

In [ ]:
ir_powers_new=[]
for files in os.listdir(path+"\\Exp_Data\\975nm_new"):
    file_list = files[len(f"{'975'}_"):-2]
    if len(file_list)<=5:
        ir_powers_new.append(int(file_list))
ir_powers_new=np.sort(np.array(ir_powers_new))

for i in irs:
    for j in ir_powers_new:
        globals()['current_'+f'{i}_'+f'{j}'] =[]
        for files in os.listdir(path+f"\\Exp_Data\\{i}nm_new\\{i}_{j}mW"):
            file_list = files[:-4]
            if len(file_list)<=5:
                globals()['current_'+f'{i}_'+f'{j}'].append(float(file_list))
        globals()['current_'+f'{i}_'+f'{j}']= np.sort(np.array(globals()['current_'+f'{i}_'+f'{j}'][1:]))                 #starting from index 1 to ignore the first data
        globals()['power_'+f'{i}_'+f'{j}']= fun.interpolation(current_new,power_new,globals()['current_'+f'{i}_'+f'{j}'])     #like : power_1064_100

current_green = []
for files in os.listdir(path+f"\\Exp_Data\\green_new"):
    file_list=files[:-4]
    if len(file_list)<=5:
                current_green.append(float(file_list))
current_green=np.sort(np.array(current_green))
power_green_new = fun.interpolation(current_new,power_new,current_green[1:])   #starting from index 1 to ignore the first data

In [ ]:
fig = px.scatter(x=current, y=power,labels=dict(x='Current(A)',y=f" green power (\N{greek small letter mu}W)"))
fig.update_layout(title_text="Before Interpolation")
fig.show()

###### Second DataSets

In [ ]:
for i in irs:
    globals()['power_'+f'{i}'+'_new']=[]
    globals()['data_'+f'{i}'+'_new']=[]
    globals()['intensity_'+f'{i}'+'_new']=[]
    globals()['ratio_'+f'{i}'+'_new']=[]
    for j in ir_powers_new:
        txt_files = glob.glob(path+f"\\Exp_Data\\{i}nm_new\\{i}_{j}mW\\*.txt")
        globals()['data_'+f'{i}_'+f'{j}'] = fun.background(fun.extract_data(txt_files,window=globals()['window_'+f'{i}']),upto=50)           #like : data_1064_20     with corrections

        globals()['intensity_'+f'{i}_'+f'{j}'] = fun.intensity(globals()['data_'+f'{i}_'+f'{j}'],globals()['power_'+f'{i}_'+f'{j}'],cutOff,pl_range=pl_range,dim=2)[0]        #like : intensity_1064_100
        globals()['ratio_'+f'{i}_'+f'{j}'] = fun.intensity(globals()['data_'+f'{i}_'+f'{j}'],globals()['power_'+f'{i}_'+f'{j}'],cutOff,pl_range=pl_range,dim=2)[1]            #like : ratio_1064_100

        globals()['power_'+f'{i}'+'_new'].append(globals()['power_'+f'{i}_'+f'{j}'])                    #like power_1064_new
        globals()['data_'+f'{i}'+'_new'].append(globals()['data_'+f'{i}_'+f'{j}'])                      #like data_1064_new
        globals()['intensity_'+f'{i}'+'_new'].append(globals()['intensity_'+f'{i}_'+f'{j}'])            #like intensity_1064_new
        globals()['ratio_'+f'{i}'+'_new'].append(globals()['ratio_'+f'{i}_'+f'{j}'])                    #like ratio_1064_new

#verification
# np.array_equal(data_1064_20mw,data_1064_20_)

#only green dataSets
txt_files = glob.glob(path+"\\Exp_Data\\green_new\\*.txt")
data_green_new = fun.background(fun.extract_data(txt_files,window=window_green),upto=50)
intensity_green_new,ratio_green_new = fun.intensity(data_green_new,power_green_new,cutOff,pl_range=pl_range,dim=2)

In [ ]:
for i in ir_powers_new:
    globals()['scale_'+f'{irs[1]}_'+f'{i}'] = fun.laser_scaling(globals()['data_'+f'{irs[1]}_'+f'{i}'],globals()['power_'+f'{irs[1]}_'+f'{i}'],scale_window)     #like scale_1064_100
    print(globals()['scale_'+f'{irs[1]}_'+f'{i}'])

##### <span style='color: Red'> Calibration of <span style='color: Blue'>532nm</span> laser</span> for third dataSets (Only 975nm and reference Green)
>> calibration is taken same as the second dataSets

In [ ]:
#refernces
irs = ['975']
scale_window = np.array([525,535])  #window to get the scaling of intensity with laser_power
current_new = np.array([2.3,2.18,2.1,2.011,1.8,1.521,1.4,1.2,1.1,1,0.8,0.627])
power_new = np.array([68,84,82.7,66,46,36,27,24,19,22,17,8.5])
window_975 = np.array([686,704])  # Dip_correcton window
window_green = np.array([690,703])
# window_975 = 'False'
# window_green = 'False'
window_green_scaling = 50

In [ ]:
ir_powers_new_2=[]
for files in os.listdir(path+"\\Exp_Data\\975nm_new_2"):
    file_list = files[len(f"{'975'}_"):-2]
    if len(file_list)<=5:
        ir_powers_new_2.append(int(file_list))
ir_powers_new_2=np.sort(np.array(ir_powers_new_2))

for i in irs:
    for j in ir_powers_new_2:
        globals()['current_'+f'{i}_'+f'{j}_2'] =[]
        for files in os.listdir(path+f"\\Exp_Data\\{i}nm_new_2\\{i}_{j}mW"):
            file_list = files[:-4]
            if len(file_list)<=5:
                globals()['current_'+f'{i}_'+f'{j}_2'].append(float(file_list))
        globals()['current_'+f'{i}_'+f'{j}_2']= np.sort(np.array(globals()['current_'+f'{i}_'+f'{j}_2'][1:]))                 #starting from index 1 to ignore the first data
        globals()['power_'+f'{i}_'+f'{j}_2']= fun.interpolation(current_new,power_new,globals()['current_'+f'{i}_'+f'{j}_2'])     #like : power_975_100_2

current_green_2 = []
for files in os.listdir(path+f"\\Exp_Data\\green_new_2"):
    file_list=files[:-4]
    if len(file_list)<=5:
                current_green_2.append(float(file_list))
current_green_2=np.sort(np.array(current_green_2))
power_green_new_2 = fun.interpolation(current_new,power_new,current_green_2[1:])

current_green_3 = []
for files in os.listdir(path+f"\\Exp_Data\\green_new_3"):
    file_list=files[:-4]
    if len(file_list)<=5:
                current_green_3.append(float(file_list))
current_green_3=np.sort(np.array(current_green_3))
power_green_new_3 = fun.interpolation(current_new,power_new,current_green_3[1:])

current_green_50_20 = []     #green regerence for only 50mW and 20mW
for files in os.listdir(path+f"\\Exp_Data\\green_new_50_20"):
    file_list=files[:-4]
    if len(file_list)<=5:
                current_green_50_20.append(float(file_list))
current_green_50_20=np.sort(np.array(current_green_50_20))
power_green_new_50_20 = fun.interpolation(current_new,power_new,current_green_50_20[1:])

In [ ]:
for i in irs:
    globals()['power_'+f'{i}'+'_new_2']=[]
    globals()['data_'+f'{i}'+'_new_2']=[]
    globals()['intensity_'+f'{i}'+'_new_2']=[]
    globals()['ratio_'+f'{i}'+'_new_2']=[]
    for j in ir_powers_new_2:
        txt_files = glob.glob(path+f"\\Exp_Data\\{i}nm_new_2\\{i}_{j}mW\\*.txt")
        globals()['data_'+f'{i}_'+f'{j}_2'] = fun.background(fun.extract_data(txt_files,window=globals()['window_'+f'{i}']),upto=50)           #like : data_975_20_2   with corrections

        globals()['intensity_'+f'{i}_'+f'{j}_2'] = fun.intensity(globals()['data_'+f'{i}_'+f'{j}_2'],globals()['power_'+f'{i}_'+f'{j}_2'],cutOff,pl_range=pl_range,dim=2)[0]        #like : intensity_975_100_2
        globals()['ratio_'+f'{i}_'+f'{j}_2'] = fun.intensity(globals()['data_'+f'{i}_'+f'{j}_2'],globals()['power_'+f'{i}_'+f'{j}_2'],cutOff,pl_range=pl_range,dim=2)[1]            #like : ratio_975_100_2

        globals()['power_'+f'{i}'+'_new_2'].append(globals()['power_'+f'{i}_'+f'{j}_2'])                    #like power_975_new_2
        globals()['data_'+f'{i}'+'_new_2'].append(globals()['data_'+f'{i}_'+f'{j}_2'])                      #like data_975_new_2
        globals()['intensity_'+f'{i}'+'_new_2'].append(globals()['intensity_'+f'{i}_'+f'{j}_2'])            #like intensity_975_new_2
        globals()['ratio_'+f'{i}'+'_new_2'].append(globals()['ratio_'+f'{i}_'+f'{j}_2'])                    #like ratio_975_new_2

#only green dataSets
txt_files = glob.glob(path+"\\Exp_Data\\green_new_2\\*.txt")
data_green_new_2 = fun.background(fun.extract_data(txt_files,window=window_green),upto=50)
intensity_green_new_2,ratio_green_new_2 = fun.intensity(data_green_new_2,power_green_new_2,cutOff,pl_range=pl_range,dim=2)

#only green dataSets
txt_files = glob.glob(path+"\\Exp_Data\\green_new_3\\*.txt")
data_green_new_3 = fun.background(fun.extract_data(txt_files,window=window_green),upto=50)
intensity_green_new_3,ratio_green_new_3 = fun.intensity(data_green_new_3,power_green_new_3,cutOff,pl_range=pl_range,dim=2)

#only green dataSets
txt_files = glob.glob(path+"\\Exp_Data\\green_new_50_20\\*.txt")
data_green_new_50_20 = fun.background(fun.extract_data(txt_files,window=window_green),upto=50)
intensity_green_new_50_20,ratio_green_new_50_20 = fun.intensity(data_green_new_50_20,power_green_new_50_20,cutOff,pl_range=pl_range,dim=2)

##### Fourth DataSets

In [ ]:
#refernces
irs = ['975']
scale_window = np.array([525,535])  #window to get the scaling of intensity with laser_power

#calibration
current_four = np.array([2.3,2.18,2.1,2.011,1.8,1.521,1.4,1.2,1.1,1,0.8,0.627])
power_four = np.array([68,84,82.7,66,46,36,27,24,19,22,17,8.5])

# Dip_correcton window
window_975 = np.array([655,668])
window_green = np.array([655,668])
# window_975 = 'False'
# window_green = 'False'

#getting the ir_powers from the folder
ir_powers_four=[]
for files in os.listdir(path+"\\Exp_Data\\975nm_four"):
    file_list = files[len(f"{'975'}_"):-2]
    if len(file_list)<=5:
        ir_powers_four.append(int(file_list))
ir_powers_four=np.sort(np.array(ir_powers_four))


#for each IR, getting the Green Power values from Current values by interpolation  : Data should not contain names like G_
for i in irs:
    for j in ir_powers_four:
        globals()['current_'+f'{i}_'+f'{j}'] =[]
        for files in os.listdir(path+f"\\Exp_Data\\{i}nm_four\\{i}_{j}mW"):
            file_list = files[:-4]
            if len(file_list)<=5:
                globals()['current_'+f'{i}_'+f'{j}'].append(float(file_list))
        globals()['current_'+f'{i}_'+f'{j}']= np.sort(np.array(globals()['current_'+f'{i}_'+f'{j}'][1:]))                 #starting from index 1 to ignore the first data
        globals()['power_'+f'{i}_'+f'{j}']= fun.interpolation(current_four,power_four,globals()['current_'+f'{i}_'+f'{j}'])     #like : power_1064_100

#getting the power values for Only Green
current_green = []
for files in os.listdir(path+f"\\Exp_Data\\green_four"):
    file_list=files[:-4]
    if len(file_list)<=5:
                current_green.append(float(file_list))
current_green=np.sort(np.array(current_green))
power_green_four = fun.interpolation(current_four,power_four,current_green[1:])     #starting from index 1 to ignore the first data


#Extracting the DataSets
for i in irs:
    globals()['power_'+f'{i}'+'_four']=[]
    globals()['data_'+f'{i}'+'_four']=[]
    globals()['intensity_'+f'{i}'+'_four']=[]
    globals()['ratio_'+f'{i}'+'_four']=[]
    for j in ir_powers_four:
        txt_files = glob.glob(path+f"\\Exp_Data\\{i}nm_four\\{i}_{j}mW\\*.txt")
        globals()['data_'+f'{i}_'+f'{j}'] = fun.background(fun.extract_data(txt_files,window=globals()['window_'+f'{i}']),upto=50)           #like : data_975_20     with corrections

        globals()['intensity_'+f'{i}_'+f'{j}'] = fun.intensity(globals()['data_'+f'{i}_'+f'{j}'],globals()['power_'+f'{i}_'+f'{j}'],cutOff,pl_range=pl_range,dim=2)[0]        #like : intensity_975_100
        globals()['ratio_'+f'{i}_'+f'{j}'] = fun.intensity(globals()['data_'+f'{i}_'+f'{j}'],globals()['power_'+f'{i}_'+f'{j}'],cutOff,pl_range=pl_range,dim=2)[1]            #like : ratio_975_100

        globals()['power_'+f'{i}'+'_four'].append(globals()['power_'+f'{i}_'+f'{j}'])                    #like power_975_four
        globals()['data_'+f'{i}'+'_four'].append(globals()['data_'+f'{i}_'+f'{j}'])                      #like data_975_four
        globals()['intensity_'+f'{i}'+'_four'].append(globals()['intensity_'+f'{i}_'+f'{j}'])            #like intensity_975_four
        globals()['ratio_'+f'{i}'+'_four'].append(globals()['ratio_'+f'{i}_'+f'{j}'])                    #like ratio_975_four


#only green dataSets
txt_files = glob.glob(path+"\\Exp_Data\\green_four\\*.txt")
data_green_four = fun.background(fun.extract_data(txt_files,window=window_green),upto=50)
intensity_green_four,ratio_green_four = fun.intensity(data_green_four,power_green_four,cutOff,pl_range=pl_range,dim=2)

In [ ]:
#for detector scaling
txt_files = glob.glob(path+f"\\Exp_Data\\532laser_spectra\\*.txt")
data_laser_four = fun.background(fun.extract_data(txt_files,window=False),upto=10)
laser_four=fun.laser_scaling(data_laser_four,power_green_four,np.array([520,540]))

In [ ]:
data_green_four.shape,power_green_four.shape

##### Error DataSets

In [ ]:
def sort_files(path):
    txt_files = glob.glob(path+"\\*.txt")

    #sorting the txt_files
    file_list=[]
    for files in os.listdir(path)[0:]:
        if len(files[:-4])<=5:
            file_list.append(int(files[:-4]))
    file_list=np.array(file_list)
    txt_files_renum=[]
    for i in range(len(file_list)):
        i+=1
        j=np.where(file_list==i)[0][0]
        txt_files_renum.append(txt_files[j])

    return txt_files_renum

In [ ]:
#getting the number values for error Green
num_error = []
for files in os.listdir(path+f"\\Exp_Data\\532nm_error_bars"):
    file_list=files[:-4]
    if len(file_list)<=5:
                num_error.append(float(file_list))
num_error=np.sort(np.array(num_error[1:]))

num_error2 = []
for files in os.listdir(path+f"\\Exp_Data\\532nm_error_bars2"):
    file_list=files[:-4]
    if len(file_list)<=5:
                num_error2.append(float(file_list))
num_error2=np.sort(np.array(num_error2[1:]))


window_error = np.array([654,664])

#extracting from the sorted txt files
txt_files = sort_files(path+"\\Exp_Data\\532nm_error_bars")
pl_error = fun.background(fun.extract_data(txt_files,window=window_error),upto=50)

txt_files2 = sort_files(path+"\\Exp_Data\\532nm_error_bars2")
pl_error2 = fun.background(fun.extract_data(txt_files2,window=window_error),upto=50)

intensity_error,ratio_error = fun.intensity(pl_error,num_error,cutOff,pl_range=pl_range,dim=2)
intensity_error2,ratio_error2 = fun.intensity(pl_error2,num_error2,cutOff,pl_range=pl_range,dim=2)

laser_error=fun.laser_scaling(pl_error,num_error,np.array([520,540]))
laser_error2=fun.laser_scaling(pl_error2,num_error2,np.array([520,540]))

##### Combined DataSets

In [ ]:
#green and 1064nm from the second datasets
# power_green_new,intensity_green_new,ratio_green_new

# power_1064_new=np.array([power_1064_100,power_1064_100,power_1064_100,power_1064_100])
# data_1064_new=np.array([data_1064_100,data_1064_100,data_1064_100,data_1064_100])
# intensity_1064_new=np.array([intensity_1064_100,intensity_1064_100,intensity_1064_100,intensity_1064_100])
# ratio_1064_new=np.array([ratio_1064_100,ratio_1064_100,ratio_1064_100,ratio_1064_100])

#green and 975nm from the fourth datasets
# power_green_four,intensity_green_four,ratio_green_four

# power_975_four=np.array([power_975_100,power_975_100,power_975_100,power_975_100])
# data_975_four=np.array([data_975_100,data_975_100,data_975_100,data_975_100])
# intensity_975_four=np.array([intensity_975_100,intensity_975_100,intensity_975_100,intensity_975_100])
# ratio_975_four=np.array([ratio_975_100,ratio_975_100,ratio_975_100,ratio_975_100])

#### Experimental Plots

##### First set of Data

In [ ]:
# fig = go.Figure()
# for i in range(data_green.shape[1]-1):
#     fig.add_scatter(x=data_green[:,0],y=data_green[:,i+1],name=f" green power {greenOnly_powers[i]}  \N{greek small letter mu}W")
# fig.update_layout(xaxis = dict(title = 'Wavelenght (nm)'),yaxis = dict(title = 'PL of 532nm'))
# fig

In [ ]:
# fig = go.Figure()
# for i in range(data_1064[0].shape[1]-1):
#     fig.add_scatter(x=data_1064[0][:,0],y=data_1064[0][:,i+1],name=f" green power {green_powers[i]}  \N{greek small letter mu}W")
# fig.update_layout(xaxis = dict(title = 'Wavelenght (nm)'),yaxis = dict(title = 'PL of 1064nm'))
# fig

In [ ]:
# fig = go.Figure()
# for i in range(data_975[0].shape[1]-1):   # data for green powers 43 and 45 is exactly same as I have copied one data to another due to lack of that green power and to match with 1064nm ir
#     fig.add_scatter(x=data_975[0][:,0],y=data_975[0][:,i+1],name=f" green power {green_powers[i]}  \N{greek small letter mu}W")
# fig.update_layout(xaxis = dict(title = 'Wavelength (nm)'),yaxis = dict(title= 'PL of 975nm'))
# fig

In [ ]:
# fig = go.Figure()
# fig.add_scatter(x=greenOnly_powers,y=intensity_green,name='Only Green')
# for i in range(data_1064.shape[0]):
#     fig.add_scatter(x=green_powers,y=intensity_1064[i,:],name=f'1064nm power {ir_powers[i]} mW ')
#     fig.add_scatter(x=green_powers,y=intensity_975[i,:],name=f'975nm power {ir_powers[i]} mW ')
# fig.update_layout(xaxis = dict(title = 'Green Power (\N{greek small letter mu}W)'),yaxis = dict(title = 'Integrated PL' ))
# fig

In [ ]:
# approx_ratio_green_1=np.mean(ratio_green)
# approx_ratio_1064_1=[];approx_ratio_975_1=[]
# for i in range(len(power_1064_new)):
#     approx_ratio_1064_1.append(np.mean(ratio_1064[i]));    approx_ratio_975_1.append(np.mean(ratio_975[i]))
# approx_ratio_1064_1=np.mean(np.array(approx_ratio_1064_1));  approx_ratio_975_1=np.mean(np.array(approx_ratio_975_1))
# print(f'Ratios from Set1: (532nm,{np.round(approx_ratio_green_1,4)})  (1064nm,{np.round(approx_ratio_1064_1,4)})  (975nm,{np.round(approx_ratio_975_1,4)})')

In [ ]:
# ir_powers

In [ ]:
# fig = go.Figure()
# fig.add_scatter(x=green_powers,y=np.repeat(approx_ratio_green_1,green_powers.shape[0]),name='Approx. ratio Green')
# fig.add_scatter(x=ir_powers,y=np.repeat(approx_ratio_1064_1,ir_powers.shape[0]),name='Approx. ratio 1064nm')
# fig.add_scatter(x=ir_powers,y=np.repeat(approx_ratio_975_1,ir_powers.shape[0]),name='Approx. ratio 975nm')
# fig.add_scatter(x=green_powers,y=ratio_green,name='Only Green New',mode="markers",)
# for i in range(len(ir_powers)):
#     fig.add_scatter(x=green_powers,y=ratio_1064[i],name=f'1064nm power new : {ir_powers [i]} mW ',mode="markers",)
#     fig.add_scatter(x=green_powers,y=ratio_975[i],name=f'975nm power new : {ir_powers [i]} mW ',mode="markers",)
# fig.update_layout(xaxis = dict(title = 'Green Power (\N{greek small letter mu}W)'),yaxis = dict(title = 'Charge State Ratio' ))
# fig

In [ ]:
# fig = go.Figure()
# fig.add_scatter(x=green_powers,y=ratio_green,name='Green only')
# for i in range(data_1064.shape[0]):
#     fig.add_scatter(x=green_powers,y=ratio_1064[i,:],name=f'1064nm power {ir_powers[i]} mW ')
#     fig.add_scatter(x=green_powers,y=ratio_975[i,:],name=f'975nm power {ir_powers[i]} mW ')
# fig.update_layout(xaxis = dict(title = 'Green Power (\N{greek small letter mu}W)'),yaxis = dict(title = "Charge State Ratio" ))
# fig

##### Second set of Data

In [ ]:
fig = go.Figure()
for i in range(data_green_new.shape[1]-1):
    fig.add_scatter(x=data_green_new[:,0],y=data_green_new[:,i+1],name=f'Green power : {np.round(power_green_new[i],3)} \N{greek small letter mu}W')
fig.update_layout(xaxis = dict(title = 'Wavelenght (nm)'),yaxis = dict(title = 'PL of 532nm (Second Set)'))
fig

In [ ]:
fig = go.Figure()
fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.10)
for j in range(len(ir_powers_new)):
    for i in range(globals()['data_1064_'+f'{ir_powers_new[j]}'].shape[1]-1):
        fig.add_scatter(x=globals()['data_1064_'+f'{ir_powers_new[j]}'][:,0],y=globals()['data_1064_'+f'{ir_powers_new[j]}'][:,i+1] ,
                        name=f'IR_'+f'{ir_powers_new[j]}mW_Green : {np.round(power_1064_new[j][i],3)} \N{greek small letter mu}W',row=j+1, col=1)
        fig.update_xaxes(title_text=f"Wavelenght", row=j+1, col=1)
        fig.update_yaxes(title_text=f"PL_1064nm_{ir_powers_new[j]}mW", row=j+1, col=1)
fig.update_layout(height=1300, width=1300,
                  title_text="Second PL Set of 1064nm")
fig

In [ ]:
power_975_new

In [ ]:
fig = go.Figure()
fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.10)
for j in range(len(ir_powers_new)):
    for i in range(globals()['data_975_'+f'{ir_powers_new[j]}'].shape[1]-1):
        fig.add_scatter(x=globals()['data_975_'+f'{ir_powers_new[j]}'][:,0],y=globals()['data_975_'+f'{ir_powers_new[j]}'][:,i+1],
                        name=f'IR_'+f'{ir_powers_new[j]}mW_Green : {np.round(power_975_new[j][i],3)} \N{greek small letter mu}W', row=j+1, col=1)
        fig.update_xaxes(title_text=f"Wavelenght", row=j+1, col=1)
        fig.update_yaxes(title_text=f"PL_975nm_{ir_powers_new[j]}mW", row=j+1, col=1)
fig.update_layout(height=1300, width=1300,
                  title_text="Second PL Set of 975nm")
fig

In [ ]:
np.array_equal(intensity_1064_new[0],intensity_1064_20)

In [ ]:
fig = go.Figure()
fig.add_scatter(x=power_green_new,y=intensity_green_new,name='Only Green New',mode="markers",)
for i in range(len(power_1064_new)):
    fig.add_scatter(x=power_1064_new[i],y=intensity_1064_new[i],name=f'1064nm power new : {ir_powers_new [i]} mW ',mode="markers",)
    # fig.add_scatter(x=power_975_new[i],y=intensity_975_new[i],name=f'975nm power new : {ir_powers_new [i]} mW ',mode="markers",)
fig.update_layout(xaxis = dict(title = 'Green Power (\N{greek small letter mu}W)'),yaxis = dict(title = 'Integrated PL' ))
fig

In [ ]:
approx_ratio_green=np.mean(ratio_green_new)
approx_ratio_1064=[];approx_ratio_975=[]
for i in range(len(power_1064_new)):
    approx_ratio_1064.append(np.mean(ratio_1064_new[i]));    approx_ratio_975.append(np.mean(ratio_975_new[i]))
approx_ratio_1064=np.mean(np.array(approx_ratio_1064));  approx_ratio_975=np.mean(np.array(approx_ratio_975))
print(f'Ratios from Set2: (532nm,{np.round(approx_ratio_green,4)})  (1064nm,{np.round(approx_ratio_1064,4)})  (975nm,{np.round(approx_ratio_975,4)})')

In [ ]:
fig = go.Figure()
fig.add_scatter(x=power_green_new,y=np.repeat(approx_ratio_green,power_green_new.shape[0]),name='Approx. ratio Green New')
fig.add_scatter(x=power_1064_new[0],y=np.repeat(approx_ratio_1064,power_1064_new[0].shape[0]),name='Approx. ratio 1064nm New')
fig.add_scatter(x=power_975_new[0],y=np.repeat(approx_ratio_975,power_975_new[0].shape[0]),name='Approx. ratio 975nm New')
fig.add_scatter(x=power_green_new,y=ratio_green_new,name='Only Green New',mode="markers",)
for i in range(len(power_1064_new)):
    fig.add_scatter(x=power_1064_new[i],y=ratio_1064_new[i],name=f'1064nm power new : {ir_powers_new [i]} mW ',mode="markers",)
    fig.add_scatter(x=power_975_new[i],y=ratio_975_new[i],name=f'975nm power new : {ir_powers_new [i]} mW ',mode="markers",)
fig.update_layout(xaxis = dict(title = 'Green Power (\N{greek small letter mu}W)'),yaxis = dict(title = 'Charge State Ratio' ))
fig

##### Third set of Data

In [ ]:
fig = go.Figure()
fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.10)
for j in range(len(ir_powers_new_2)):
    for i in range(globals()['data_975_'+f'{ir_powers_new_2[j]}_2'].shape[1]-1):
        fig.add_scatter(x=globals()['data_975_'+f'{ir_powers_new_2[j]}_2'][:,0],y=globals()['data_975_'+f'{ir_powers_new_2[j]}_2'][:,i+1],
                        name=f'IR_'+f'{ir_powers_new_2[j]}mW_Green : {np.round(power_975_new_2[j][i],3)} \N{greek small letter mu}W', row=j+1, col=1)
        fig.update_xaxes(title_text=f"Wavelenght", row=j+1, col=1)
        fig.update_yaxes(title_text=f"PL_975nm_{ir_powers_new[j]}mW", row=j+1, col=1)
fig.update_layout(height=1300, width=1300,
                  title_text="Third PL Set of 975nm")
fig

In [ ]:
# fig = go.Figure()
# fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.10,
#     subplot_titles=("Set1", "Set2","Set3"))
# for i in range(data_green_new_2.shape[1]-1):
#     fig.add_scatter(x=data_green_new_2[:,0],y=data_green_new_2[:,i+1],name=f'Green power_set2 : {np.round(power_green_new_2[i],3)} \N{greek small letter mu}W',row=1,col=1)
# for i in range(data_green_new_3.shape[1]-1):
#     fig.add_scatter(x=data_green_new_3[:,0],y=data_green_new_3[:,i+1],name=f'Green power_set3 : {np.round(power_green_new_3[i],3)} \N{greek small letter mu}W',row=2,col=1)
# for i in range(data_green_new_50_20.shape[1]-1):
#     fig.add_scatter(x=data_green_new_50_20[:,0],y=data_green_new_50_20[:,i+1],name=f'Green power_set_50_20 : {np.round(power_green_new_50_20[i],3)} \N{greek small letter mu}W',row=3,col=1)

# fig.update_xaxes(title_text=f"Wavelenght", row=1, col=1)
# fig.update_yaxes(title_text=f"PL_532nm_set2", row=1, col=1)
# fig.update_xaxes(title_text=f"Wavelenght", row=2, col=1)
# fig.update_yaxes(title_text=f"PL_532nm_set3", row=2, col=1)
# fig.update_xaxes(title_text=f"Wavelenght", row=3, col=1)
# fig.update_yaxes(title_text=f"PL_532nm_set_50_20", row=3, col=1)
# fig.update_layout(height=800, width=1500,
#                   title_text='PL of 532nm (Third Set)')
# fig

In [ ]:
# approx_ratio_green_2=np.mean(ratio_green_new_2)
# approx_ratio_975_2=[]
# for i in range(len(power_975_new_2)):
#     approx_ratio_975_2.append(np.mean(ratio_975_new_2[i]))
# approx_ratio_975_2=np.mean(np.array(approx_ratio_975_2))
# print(f'Ratios from Set3: (532nm,{np.round(approx_ratio_green_2,4)})  (975nm,{np.round(approx_ratio_975_2,4)})')

In [ ]:
# fig = go.Figure()
# fig.add_scatter(x=power_green_new_2,y=np.repeat(approx_ratio_green_2,power_green_new_2.shape[0]),name='Approx. ratio Green New_2')
# fig.add_scatter(x=power_975_new_2[0],y=np.repeat(approx_ratio_975_2,power_975_new_2[0].shape[0]),name='Approx. ratio 975nm New_2')
# fig.add_scatter(x=power_green_new_2,y=ratio_green_new_2,name='Only Green New_2',mode="markers",)
# for i in range(len(power_975_new_2)):
#     fig.add_scatter(x=power_975_new_2[i],y=ratio_975_new_2[i],name=f'975nm power new_2 : {ir_powers_new [i]} mW ',mode="markers",)
# fig.update_layout(xaxis = dict(title = 'Green Power (\N{greek small letter mu}W)'),yaxis = dict(title = 'Charge State Ratio' ))
# fig.show()

##### Fourth set of Data

In [ ]:
fig = go.Figure()

fig = make_subplots(rows=5, cols=1, shared_xaxes=True, vertical_spacing=0.10)
for i in range(data_green_four.shape[1]-1):
    fig.add_scatter(x=data_green_four[:,0],y=data_green_four[:,i+1],name=f'Green power : {np.round(power_green_four[i],3)} \N{greek small letter mu}W',row=1,col=1)
fig.update_xaxes(title_text= 'Wavelenght (nm)', row=1, col=1)
fig.update_yaxes(title_text='PL of 532nm', row=1, col=1)

# for j in range(len(ir_powers_four)):
#     for i in range(globals()['data_1064_'+f'{ir_powers_four[j]}'].shape[1]-1):
#         fig.add_scatter(x=globals()['data_1064_'+f'{ir_powers_four[j]}'][:,0],y=globals()['data_1064_'+f'{ir_powers_four[j]}'][:,i+1] ,
#                         name=f'IR_'+f'{ir_powers_four[j]}mW_Green : {np.round(power_1064_four[j][i],3)} \N{greek small letter mu}W',row=j+2, col=1)
#         fig.update_xaxes(title_text=f"Wavelenght", row=j+2, col=1)
#         fig.update_yaxes(title_text=f"PL_1064nm_{ir_powers_four[j]}mW", row=j+2, col=1)
# fig.update_layout(height=1300, width=1300,
#                   title_text="Fourth PL Set of 1064nm")

# fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.10)
for j in range(len(ir_powers_four)):
    for i in range(globals()['data_975_'+f'{ir_powers_four[j]}'].shape[1]-1):
        fig.add_scatter(x=globals()['data_975_'+f'{ir_powers_four[j]}'][:,0],y=globals()['data_975_'+f'{ir_powers_four[j]}'][:,i+1],
                        name=f'IR_'+f'{ir_powers_four[j]}mW_Green : {np.round(power_975_four[j][i],3)} \N{greek small letter mu}W', row=j+2, col=1)
        fig.update_xaxes(title_text=f"Wavelenght", row=j+2, col=1)
        fig.update_yaxes(title_text=f"PL_975nm_{ir_powers_four[j]}mW", row=j+2, col=1)
fig.update_layout(height=1300, width=1300,
                  title_text="Second PL Set of 975nm")

# fig.add_scatter(x=power_green_four,y=intensity_green_four,name='Only Green four',)
# for i in range(len(power_975_four)):
#     # fig.add_scatter(x=power_1064_four[i],y=intensity_1064_four[i],name=f'1064nm power four : {ir_powers_four [i]} mW ',mode="markers",)
#     fig.add_scatter(x=power_975_four[i],y=intensity_975_four[i],name=f'975nm power four : {ir_powers_four [i]} mW ',)
# for i in range(len(power_1064_new)):
#     fig.add_scatter(x=power_1064_new[i],y=intensity_1064_new[i],name=f'1064nm power new : {ir_powers_new [i]} mW ',)
# fig.update_layout(xaxis = dict(title = 'Green Power (\N{greek small letter mu}W)'),yaxis = dict(title = 'Integrated PL' ))

# approx_ratio_green=np.mean(ratio_green_four)
# approx_ratio_1064=[];approx_ratio_975=[]
# for i in range(len(power_975_four)):
#     # approx_ratio_1064.append(np.mean(ratio_1064_four[i]))
#     approx_ratio_975.append(np.mean(ratio_975_four[i]))
# # approx_ratio_1064=np.mean(np.array(approx_ratio_1064))
# approx_ratio_975=np.mean(np.array(approx_ratio_975))
# print(f'Ratios from Set2: (532nm,{np.round(approx_ratio_green,4)})  (1064nm,{np.round(approx_ratio_1064,4)})  (975nm,{np.round(approx_ratio_975,4)})')

# fig.add_scatter(x=power_green_four,y=np.repeat(approx_ratio_green,power_green_four.shape[0]),name='Approx. ratio Green four')
# fig.add_scatter(x=power_1064_four[0],y=np.repeat(approx_ratio_1064,power_1064_four[0].shape[0]),name='Approx. ratio 1064nm four')
# fig.add_scatter(x=power_975_four[0],y=np.repeat(approx_ratio_975,power_975_four[0].shape[0]),name='Approx. ratio 975nm four')
# fig.add_scatter(x=power_green_four,y=ratio_green_four,name='Only Green four',mode="markers",)
# for i in range(len(power_1064_four)):
#     fig.add_scatter(x=power_1064_four[i],y=ratio_1064_four[i],name=f'1064nm power four : {ir_powers_four [i]} mW ',mode="markers",)
#     fig.add_scatter(x=power_975_four[i],y=ratio_975_four[i],name=f'975nm power four : {ir_powers_four [i]} mW ',mode="markers",)
# fig.update_layout(xaxis = dict(title = 'Green Power (\N{greek small letter mu}W)'),yaxis = dict(title = 'Charge State Ratio' ))

fig.show()

In [ ]:
#detector_scaling
scale_four=fun.detector_scaling(data_green_four,power_green_four,np.array([528,536]))

# #scaling with refernce to green   : reference green , green of scaling set
# intensity_green_four=fun.green_scale(intensity_green_four,intensity_green_four,power_green_new,power_green_four)*intensity_green_four

# irs = ['975','1064']
# for i in irs:
#     for j in range(len(ir_powers_new)):
#         globals()[f'intensity_{i}_four'][j]=fun.green_scale(intensity_green_new,\\
#                                                             globals()[f'intensity_{i}_four'][j],power_green_new,globals()[f'power_{i}_four'][j])*globals()[f'intensity_{i}_four'][j]


#detector scaling of green
data_green_scaled_four=np.ones((data_green_four.shape))
data_green_scaled_four[:,0]=data_green_four[:,0]
data_green_scaled_four[:,1:]=data_green_four[:,1:]*scale_four

#detector scaling of 975nm
irs = ['975']

for i in irs:
    globals()[f'data_{i}_four_scaled']=[]
    for j in range(len(ir_powers_four)):
        globals()[f'scale_set_{i}_{ir_powers_four[j]}'] = fun.interpolation(power_green_four,scale_four, globals()[f'power_{i}_four'][j])
        globals()[f'data_{i}_four_scaled_{ir_powers_four[j]}']=np.ones((globals()[f'data_{i}_four'][j].shape))
        for k in range(len(globals()[f'power_{i}_four'][j])):
            globals()[f'data_{i}_four_scaled_{ir_powers_four[j]}'][:,0]=globals()[f'data_{i}_four'][j][:,0]
            globals()[f'data_{i}_four_scaled_{ir_powers_four[j]}'][:,k+1]=(globals()[f'data_{i}_four'][j][:,k+1]*globals()[f'scale_set_{i}_{ir_powers_four[j]}'][k])   #like data_975_four_scaled_100
            globals()[f'data_{i}_four_scaled'].append(globals()[f'data_{i}_four_scaled_{ir_powers_four[j]}'])                                                          #like data_975_four_scaled


##Calculating Intensity and Ratios from the DetectorScaled PLs
scaled_intensity_green_four,scaled_ratio_green_four = fun.intensity(data_green_scaled_four,power_green_four,cutOff,pl_range=pl_range,dim=2)

irs = ['975']
for i in irs:
    globals()['intensity_'+f'{i}'+'_four_scaled']=[]
    globals()['ratio_'+f'{i}'+'_four_scaled']=[]
    for j in range(len(ir_powers_four)):
        globals()['intensity_'+f'{i}_'+f'{ir_powers_four[j]}_scaled'] = fun.intensity(globals()[f'data_{i}_four_scaled_{ir_powers_four[j]}'],globals()[f'power_{i}_{ir_powers_four[j]}'],cutOff,pl_range=pl_range,dim=2)[0]        #like : intensity_975_100_scaled
        globals()['ratio_'+f'{i}_'+f'{ir_powers_four[j]}_scaled'] = fun.intensity(globals()[f'data_{i}_four_scaled_{ir_powers_four[j]}'],globals()[f'power_{i}_{ir_powers_four[j]}'],cutOff,pl_range=pl_range,dim=2)[1]            #like : ratio_975_100_scaled
        print('ratio_'+f'{i}_'+f'{ir_powers_four[j]}_scaled')
        globals()['intensity_'+f'{i}'+'_four_scaled'].append(globals()['intensity_'+f'{i}_'+f'{ir_powers_four[j]}_scaled'])            #like intensity_975_four_scaled
        globals()['ratio_'+f'{i}'+'_four_scaled'].append(np.mean(globals()['ratio_'+f'{i}_'+f'{ir_powers_four[j]}_scaled']))                    #like ratio_1975_four_scaled  with mean of each ir_powers
    globals()['ratio_'+f'{i}'+'_four_scaled']=np.array(globals()['ratio_'+f'{i}'+'_four_scaled'])


ratio_green_scaled_four=np.mean(scaled_ratio_green_four)
# ratio_975_scaled_four=np.mean(ratio_975_four_scaled)
print(f'Ratios from Detector_Scaled PLs: (532nm,{np.round(ratio_green_scaled_four,4)})  (975nm,{np.round(ratio_975_four_scaled,4)})\n')

In [ ]:
fig = go.Figure()

# fig = make_subplots(rows=5, cols=1, shared_xaxes=True, vertical_spacing=0.10)
# irs = ['975']
# for i_ir in irs:
#     for j in range(len(ir_powers_four)):
#         for i in range(globals()[f'data_{i_ir}_four_scaled_'+f'{ir_powers_four[j]}'].shape[1]-1):
#             #with correction
#             # fig.add_scatter(x=globals()[f'data_{i_ir}_four_scaled_'+f'{ir_powers_four[j]}'][:,0],y=globals()[f'data_{i_ir}_four_scaled_'+f'{ir_powers_four[j]}'][:,i+1] ,       #like data_1064_four_scaled_100
#             #                 name=f'IR_'+f'{ir_powers_four[j]}mW_Green_corrected : {np.round(globals()[f"power_{i_ir}_four"][j][i],3)} \N{greek small letter mu}W',row=j+1, col=1)
#             # # #without correction
#             fig.add_scatter(x=globals()[f'data_{i_ir}_'+f'{ir_powers_four[j]}'][:,0],y=globals()[f'data_{i_ir}_'+f'{ir_powers_four[j]}'][:,i+1] ,
#                             name=f'IR_'+f'{ir_powers_four[j]}mW_Green : {np.round(globals()[f"power_{i_ir}_four"][j][i],3)} \N{greek small letter mu}W',row=j+1, col=1)
#             fig.update_xaxes(title_text=f"Wavelenght", row=j+1, col=1)
#             fig.update_yaxes(title_text=f"PL_{i_ir}nm_{ir_powers_four[j]}mW", row=j+1, col=1)
# fig.update_layout(height=1300, width=1300,
#                   title_text=f"Detector Scaled Second PL Set of {i_ir}nm")
fig.add_scatter(x=power_green_four,y=scaled_intensity_green_four,name=f'only green four',)
for i in range(len(power_975_four)):
    fig.add_scatter(x=power_975_four[i],y=intensity_975_four_scaled[i],name=f'975nm power four : {ir_powers_four [i]} mW ',)
fig.update_layout(xaxis = dict(title = 'Green Power (\N{greek small letter mu}W)'),yaxis = dict(title = 'Integrated PL' ))

fig

##### For Errorbars

In [ ]:
fig = go.Figure()
for i in range(pl_error.shape[1]-1):
    fig.add_scatter(x=pl_error[:,0],y=pl_error[:,i+1],name=f'Number (set1) : {num_error[i]}')
    # fig.add_scatter(x=pl_error[:,0],y=pl_error[:,i+1]*np.mean(laser_error)/laser_error[i],name=f'Number (set1) : {num_error[i]}')
# for i in range(pl_error2.shape[1]-1):
#     fig.add_scatter(x=pl_error2[:,0],y=pl_error2[:,i+1],name=f'Number (after 1min break) : {num_error2[i]}')
fig.update_layout(xaxis = dict(title = 'Wavelenght (nm)'),yaxis = dict(title = 'PL of 532nm (2.17A)'))
fig

In [ ]:
fig = go.Figure()
fig.add_scatter(x=num_error,y=laser_error,name='integrated laser_error PL',mode="markers")
fig.add_scatter(x=num_error,y=intensity_error,name='Integrated PL of errors (set1)',mode="markers",)
fig.add_scatter(x=num_error,y=ratio_error,name='ratio of errors (set1)',mode="markers",)
fig.add_scatter(x=num_error2,y=intensity_error2,name='Integrated PL of errors (set2)',mode="markers",)
fig.add_scatter(x=num_error2,y=ratio_error2,name='ratio of errors (set2)',mode="markers",)
fig.show()

In [ ]:
np.mean(laser_error)

In [ ]:
std_error =np.std(intensity_error, ddof=1) /np.mean(intensity_error)

In [ ]:
std_error

In [ ]:
hist_error,x_error =np.histogram(intensity_error/np.mean(intensity_error),
                             bins=10,range=[0.75,1.15],
                            )
x_error= fun.moving_average(x_error,2)

In [ ]:
fig = go.Figure()
fig.add_bar(x=x_error,y=hist_error,name='intensity_error',)
fig.show()

##### Combined set of Data

In [ ]:
# #detector_scaling
scale_set3=fun.detector_scaling(data_green_new_3,power_green_new_3,np.array([528,536]))

###### Correction with Green-scale and Detector_scale

In [ ]:
# #scaling with refernce to green
# ### first window is also taken for scaling
intensity_green_new_3=fun.green_scale(intensity_green_new,intensity_green_new_3,power_green_new,power_green_new_3)*intensity_green_new_3
# intensity_green_new_2=fun.green_scale(intensity_green_new,intensity_green_new_2,power_green_new,power_green_new_2)*intensity_green_new_2
# intensity_green_new_50_20=fun.green_scale(intensity_green_new,intensity_green_new_50_20,power_green_new,power_green_new_50_20)*intensity_green_new_50_20

# intensity_975_new_2[3]=fun.green_scale(intensity_green_new,intensity_green_new_3,power_green_new,power_green_new_3)*intensity_975_new_2[3]    ##intensity_green_new_2 is not used
# intensity_975_new_2[2]=fun.green_scale(intensity_green_new,intensity_green_new_3,power_green_new,power_green_new_3)*intensity_975_new_2[2]
# intensity_975_new_2[1]=fun.green_scale(intensity_green_new,intensity_green_new_50_20,power_green_new,power_green_new_50_20)*intensity_975_new_2[1]
# intensity_975_new_2[0]=fun.green_scale(intensity_green_new,intensity_green_new_50_20,power_green_new,power_green_new_50_20)*intensity_975_new_2[0]

In [ ]:
# #detector scaling of green
# data_green_scaled=np.ones((data_green_new_3.shape))
# data_green_scaled[:,0]=data_green_new_3[:,0]
# data_green_scaled[:,1:]=data_green_new_3[:,1:]*scale_set3

In [ ]:
# #detector scaling of 1064nm
# irs = ['1064']

# for i in irs:
#     globals()[f'data_{i}_new_scaled']=[]
#     for j in range(len(ir_powers_new)):
#         globals()[f'scale_set_{i}_{ir_powers_new[j]}'] = fun.interpolation(power_green_new_3,scale_set3, globals()[f'power_{i}_new'][j])
#         globals()[f'data_{i}_new_scaled_{ir_powers_new[j]}']=np.ones((globals()[f'data_{i}_new'][j].shape))
#         for k in range(len(globals()[f'power_{i}_new'][j])):
#             globals()[f'data_{i}_new_scaled_{ir_powers_new[j]}'][:,0]=globals()[f'data_{i}_new'][j][:,0]
#             globals()[f'data_{i}_new_scaled_{ir_powers_new[j]}'][:,k+1]=(globals()[f'data_{i}_new'][j][:,k+1]*globals()[f'scale_set_{i}_{ir_powers_new[j]}'][k])   #like data_1064_new_scaled_100
#             globals()[f'data_{i}_new_scaled'].append(globals()[f'data_{i}_new_scaled_{ir_powers_new[j]}'])                                                          #like data_1064_new_scaled

In [ ]:
# #detector scaling of 975nm
# irs = ['975']

# for i in irs:
#     globals()[f'data_{i}_new_scaled']=[]
#     for j in range(len(ir_powers_new)-2):  #for 20 and 50mW
#         globals()[f'scale_set_{i}_{ir_powers_new[j]}'] = fun.interpolation(power_green_new_3,scale_set3, globals()[f'power_{i}_new'][j])
#         globals()[f'data_{i}_new_scaled_{ir_powers_new[j]}']=np.ones((globals()[f'data_{i}_new'][j].shape))
#         for k in range(len(globals()[f'power_{i}_new'][j])):
#             globals()[f'data_{i}_new_scaled_{ir_powers_new[j]}'][:,0]=globals()[f'data_{i}_new'][j][:,0]
#             globals()[f'data_{i}_new_scaled_{ir_powers_new[j]}'][:,k+1]=(globals()[f'data_{i}_new'][j][:,k+1]*globals()[f'scale_set_{i}_{ir_powers_new[j]}'][k])   #like data_975_new_scaled_50
#             globals()[f'data_{i}_new_scaled'].append(globals()[f'data_{i}_new_scaled_{ir_powers_new[j]}'])                                                          #like data_975_new_scaled
#     for j in range(len(ir_powers_new)-2): #for 70 and 100mW
#         j+=2
#         globals()[f'scale_set_{i}_{ir_powers_new[j]}'] = fun.interpolation(power_green_new_3,scale_set3, globals()[f'power_{i}_new_2'][j])
#         globals()[f'data_{i}_new_scaled_{ir_powers_new[j]}']=np.ones((globals()[f'data_{i}_new_2'][j].shape))
#         for k in range(len(globals()[f'power_{i}_new_2'][j])):
#             globals()[f'data_{i}_new_scaled_{ir_powers_new[j]}'][:,0]=globals()[f'data_{i}_new_2'][j][:,0]
#             globals()[f'data_{i}_new_scaled_{ir_powers_new[j]}'][:,k+1]=(globals()[f'data_{i}_new_2'][j][:,k+1]*globals()[f'scale_set_{i}_{ir_powers_new[j]}'][k])
#             globals()[f'data_{i}_new_scaled'].append(globals()[f'data_{i}_new_scaled_{ir_powers_new[j]}'])

In [ ]:
# ##Calculating Intensity and Ratios from the DetectorScaled PLs
# scaled_intensity_green,scaled_ratio_green = fun.intensity(data_green_scaled,power_green_new_3,cutOff,pl_range=pl_range,dim=2)

# irs = ['1064']
# for i in irs:
#     globals()['intensity_'+f'{i}'+'_new_scaled']=[]
#     globals()['ratio_'+f'{i}'+'_new_scaled']=[]
#     for j in range(len(ir_powers_new)):
#         globals()['intensity_'+f'{i}_'+f'{ir_powers_new[j]}_scaled'] = fun.intensity(globals()[f'data_{i}_new_scaled_{ir_powers_new[j]}'],globals()[f'power_{i}_{ir_powers_new[j]}'],cutOff,pl_range=pl_range,dim=2)[0]        #like : intensity_1064_100_scaled
#         globals()['ratio_'+f'{i}_'+f'{ir_powers_new[j]}_scaled'] = fun.intensity(globals()[f'data_{i}_new_scaled_{ir_powers_new[j]}'],globals()[f'power_{i}_{ir_powers_new[j]}'],cutOff,pl_range=pl_range,dim=2)[1]            #like : ratio_1064_100_scaled
#         print('ratio_'+f'{i}_'+f'{ir_powers_new[j]}_scaled')
#         globals()['intensity_'+f'{i}'+'_new_scaled'].append(globals()['intensity_'+f'{i}_'+f'{ir_powers_new[j]}_scaled'])            #like intensity_1064_new_scaled
#         globals()['ratio_'+f'{i}'+'_new_scaled'].append(np.mean(globals()['ratio_'+f'{i}_'+f'{ir_powers_new[j]}_scaled']))                    #like ratio_1064_new_scaled  with mean of each ir_powers
#     globals()['ratio_'+f'{i}'+'_new_scaled']=np.array(globals()['ratio_'+f'{i}'+'_new_scaled'])


# irs = ['975']
# for i in irs:
#     globals()['intensity_'+f'{i}'+'_new_scaled']=[]
#     globals()['ratio_'+f'{i}'+'_new_scaled']=[]
#     for j in range(len(ir_powers_new)-2):
#         globals()['intensity_'+f'{i}_'+f'{ir_powers_new[j]}_scaled'] = fun.intensity(globals()[f'data_{i}_new_scaled_{ir_powers_new[j]}'],globals()[f'power_{i}_{ir_powers_new[j]}'],cutOff,pl_range=pl_range,dim=2)[0]        #like : intensity_1064_100_scaled
#         globals()['ratio_'+f'{i}_'+f'{ir_powers_new[j]}_scaled'] = fun.intensity(globals()[f'data_{i}_new_scaled_{ir_powers_new[j]}'],globals()[f'power_{i}_{ir_powers_new[j]}'],cutOff,pl_range=pl_range,dim=2)[1]            #like : ratio_1064_100_scaled
#         globals()['intensity_'+f'{i}'+'_new_scaled'].append(globals()['intensity_'+f'{i}_'+f'{ir_powers_new[j]}_scaled'])                    #like intensity_1064_new_scaled
#         globals()['ratio_'+f'{i}'+'_new_scaled'].append(np.mean(globals()['ratio_'+f'{i}_'+f'{ir_powers_new[j]}_scaled']))                    #like ratio_1064_new_scaled  with mean of each ir_powers
#     for j in range(len(ir_powers_new)-2):
#         j+=2
#         globals()['intensity_'+f'{i}_'+f'{ir_powers_new[j]}_scaled'] = fun.intensity(globals()[f'data_{i}_new_scaled_{ir_powers_new[j]}'],globals()[f'power_{i}_{ir_powers_new[j]}_2'],cutOff,pl_range=pl_range,dim=2)[0]        #like : intensity_1064_100_scaled
#         globals()['ratio_'+f'{i}_'+f'{ir_powers_new[j]}_scaled'] = fun.intensity(globals()[f'data_{i}_new_scaled_{ir_powers_new[j]}'],globals()[f'power_{i}_{ir_powers_new[j]}_2'],cutOff,pl_range=pl_range,dim=2)[1]            #like : ratio_1064_100_scaled
#         globals()['intensity_'+f'{i}'+'_new_scaled'].append(globals()['intensity_'+f'{i}_'+f'{ir_powers_new[j]}_scaled'])            #like intensity_1064_new_scaled
#         globals()['ratio_'+f'{i}'+'_new_scaled'].append(np.mean(globals()['ratio_'+f'{i}_'+f'{ir_powers_new[j]}_scaled']))                    #like ratio_1064_new_scaled  with mean of each ir_powers
#     globals()['ratio_'+f'{i}'+'_new_scaled']=np.array(globals()['ratio_'+f'{i}'+'_new_scaled'])

In [ ]:
# ratio_green=mean((approx_ratio_green_1,approx_ratio_green,approx_ratio_green_2))
# ratio_1064=mean((approx_ratio_1064_1,approx_ratio_1064))
# ratio_975=mean((approx_ratio_975_1,approx_ratio_975,approx_ratio_975_2))
# ratio_green_scaled=np.mean(scaled_ratio_green)
# ratio_1064_scaled=np.mean(ratio_1064_new_scaled)
# ratio_975_scaled=np.mean(ratio_975_new_scaled)

# print(f'\nRatios from average of Set1: (532nm,{np.round(approx_ratio_green_1,4)})  (975nm,{np.round(approx_ratio_975_1,4)})  (1064nm,{np.round(approx_ratio_1064_1,4)})\n')
# print(f'Ratios from average of Set2: (532nm,{np.round(approx_ratio_green,4)})  (975nm,{np.round(approx_ratio_975,4)})  (1064nm,{np.round(approx_ratio_1064,4)})\n')
# print(f'Ratios from average of Set3: (532nm,{np.round(approx_ratio_green_2,4)})  (975nm,{np.round(approx_ratio_975_2,4)})\n')
# print(f'Ratios from average of AllSets: (532nm,{np.round(ratio_green,4)})  (975nm,{np.round(ratio_975,4)})  (1064nm,{np.round(ratio_1064,4)})\n')
# print(f'Ratios from Detector_Scaled PLs: (532nm,{np.round(ratio_green_scaled,4)})  (975nm,{np.round(ratio_975_scaled,4)})  (1064nm,{np.round(ratio_1064_scaled,4)})\n')

###### Corrected Plots

In [ ]:
# fig = go.Figure()
# for i in range(data_green_new_3.shape[1]-1):
#     # fig.add_scatter(x=data_green_new_3[:,0],y=data_green_scaled[:,i+1],name=f'corrected Green power_set3 : {np.round(power_green_new_3[i],3)} \N{greek small letter mu}W') #with Correction
#     fig.add_scatter(x=data_green_new_3[:,0],y=data_green_new_3[:,i+1],name=f'uncorrected Green power_set3 : {np.round(power_green_new_3[i],3)} \N{greek small letter mu}W')  #without Correction
# fig.update_layout(title_text="Detector Scaled 532nm PL (Set 3)")
# fig.update_layout(xaxis = dict(title = f"Wavelenght (\N{greek small letter lamda})"),yaxis = dict(title = 'PhotoLuminescence' ))
# fig.show()

In [ ]:
# fig = go.Figure()
# fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.10)
# irs = ['1064']
# for i_ir in irs:
#     for j in range(len(ir_powers_new)):
#         for i in range(globals()[f'data_{i_ir}_new_scaled_'+f'{ir_powers_new[j]}'].shape[1]-1):
#             #with correction
#             # fig.add_scatter(x=globals()[f'data_{i_ir}_new_scaled_'+f'{ir_powers_new[j]}'][:,0],y=globals()[f'data_{i_ir}_new_scaled_'+f'{ir_powers_new[j]}'][:,i+1] ,       #like data_1064_new_scaled_100
#             #                 name=f'IR_'+f'{ir_powers_new[j]}mW_Green_corrected : {np.round(globals()[f"power_{i_ir}_new"][j][i],3)} \N{greek small letter mu}W',row=j+1, col=1)
#             # # #without correction
#             fig.add_scatter(x=globals()[f'data_{i_ir}_'+f'{ir_powers_new[j]}'][:,0],y=globals()[f'data_{i_ir}_'+f'{ir_powers_new[j]}'][:,i+1] ,
#                             name=f'IR_'+f'{ir_powers_new[j]}mW_Green : {np.round(globals()[f"power_{i_ir}_new"][j][i],3)} \N{greek small letter mu}W',row=j+1, col=1)
#             fig.update_xaxes(title_text=f"Wavelenght", row=j+1, col=1)
#             fig.update_yaxes(title_text=f"PL_{i_ir}nm_{ir_powers_new[j]}mW", row=j+1, col=1)
# fig.update_layout(height=1300, width=1300,
#                   title_text=f"Detector Scaled Second PL Set of {i_ir}nm")
# fig

In [ ]:
# fig = go.Figure()
# fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.10)
# irs = ['975']
# for i_ir in irs:
#     for j in range(len(ir_powers_new)-2):
#         for i in range(globals()[f'data_{i_ir}_new_scaled_'+f'{ir_powers_new[j]}'].shape[1]-1):
#             #with correction
#             # fig.add_scatter(x=globals()[f'data_{i_ir}_new_scaled_'+f'{ir_powers_new[j]}'][:,0],y=globals()[f'data_{i_ir}_new_scaled_'+f'{ir_powers_new[j]}'][:,i+1] ,
#             #                 name=f'IR_'+f'{ir_powers_new[j]}mW_Green_corrected : {np.round(globals()[f"power_{i_ir}_new"][j][i],3)} \N{greek small letter mu}W',row=j+1, col=1)
#             #without correction  : for 20 and 50mw
#             fig.add_scatter(x=globals()[f'data_{i_ir}_'+f'{ir_powers_new[j]}'][:,0],y=globals()[f'data_{i_ir}_'+f'{ir_powers_new[j]}'][:,i+1] ,
#                             name=f'IR_'+f'{ir_powers_new[j]}mW_Green : {np.round(globals()[f"power_{i_ir}_new"][j][i],3)} \N{greek small letter mu}W',row=j+1, col=1)
#             fig.update_xaxes(title_text=f"Wavelenght", row=j+1, col=1)
#             fig.update_yaxes(title_text=f"PL_{i_ir}nm_{ir_powers_new[j]}mW", row=j+1, col=1)
#     for j in range(len(ir_powers_new)-2):
#         j+=2
#         for i in range(globals()[f'data_{i_ir}_new_scaled_'+f'{ir_powers_new[j]}'].shape[1]-1):
#             #with correction
#             # fig.add_scatter(x=globals()[f'data_{i_ir}_new_scaled_'+f'{ir_powers_new[j]}'][:,0],y=globals()[f'data_{i_ir}_new_scaled_'+f'{ir_powers_new[j]}'][:,i+1] ,
#             #                 name=f'IR_'+f'{ir_powers_new[j]}mW_Green_corrected : {np.round(globals()[f"power_{i_ir}_new"][j][i],3)} \N{greek small letter mu}W',row=j+1, col=1)
#             #without correction : for 70 and 100mW
#             fig.add_scatter(x=globals()[f'data_{i_ir}_'+f'{ir_powers_new[j]}_2'][:,0],y=globals()[f'data_{i_ir}_'+f'{ir_powers_new[j]}_2'][:,i+1] ,
#                             name=f'IR_'+f'{ir_powers_new[j]}mW_Green : {np.round(globals()[f"power_{i_ir}_new"][j][i],3)} \N{greek small letter mu}W',row=j+1, col=1)
#             fig.update_xaxes(title_text=f"Wavelenght", row=j+1, col=1)
#             fig.update_yaxes(title_text=f"PL_{i_ir}nm_{ir_powers_new[j]}mW", row=j+1, col=1)
# fig.update_layout(height=1300, width=1300,
#                   title_text=f"Detector Scaled Second PL Set of {i_ir}nm")
# fig

In [ ]:
# irs=['975']
# for i in irs:
#     globals()['intensity_'+f'{i}'+'_new_scaled']=[]
#     globals()['ratio_'+f'{i}'+'_new_scaled']=[]
#     for j in range(len(ir_powers_new)-2):
#         globals()['intensity_'+f'{i}_'+f'{ir_powers_new[j]}_scaled'] = fun.intensity(globals()['data_'+f'{i}_new_scaled_'+f'{ir_powers_new[j]}'],globals()['power_'+f'{i}_'+f'{ir_powers_new[j]}'],cutOff,pl_range=pl_range,dim=2)[0]        #like : intensity_975_20_scaled
#         globals()['ratio_'+f'{i}_'+f'{ir_powers_new[j]}_scaled'] = fun.intensity(globals()['data_'+f'{i}_new_scaled_'+f'{ir_powers_new[j]}'],globals()['power_'+f'{i}_'+f'{ir_powers_new[j]}'],cutOff,pl_range=pl_range,dim=2)[1]

#         globals()['intensity_'+f'{i}'+'_new_scaled'].append(globals()['intensity_'+f'{i}_'+f'{ir_powers_new[j]}'])            #like intensity_975_new_scaled
#         globals()['ratio_'+f'{i}'+'_new_scaled'].append(globals()['ratio_'+f'{i}_'+f'{ir_powers_new[j]}'])                    #like ratio_975_new_scaled
#     for j in range(len(ir_powers_new)-2):
#         j+=2
#         globals()['intensity_'+f'{i}_'+f'{ir_powers_new[j]}_scaled'] = fun.intensity(globals()['data_'+f'{i}_new_scaled_'+f'{ir_powers_new[j]}'],globals()['power_'+f'{i}_'+f'{ir_powers_new[j]}_2'],cutOff,pl_range=pl_range,dim=2)[0]        #like : intensity_975_100_scaled
#         globals()['ratio_'+f'{i}_'+f'{ir_powers_new[j]}_scaled'] = fun.intensity(globals()['data_'+f'{i}_new_scaled_'+f'{ir_powers_new[j]}'],globals()['power_'+f'{i}_'+f'{ir_powers_new[j]}_2'],cutOff,pl_range=pl_range,dim=2)[1]

#         globals()['intensity_'+f'{i}'+'_new_scaled'].append(globals()['intensity_'+f'{i}_'+f'{ir_powers_new[j]}'])            #like intensity_975_new_scaled
#         globals()['ratio_'+f'{i}'+'_new_scaled'].append(globals()['ratio_'+f'{i}_'+f'{ir_powers_new[j]}'])                    #like ratio_975_new_scaled

In [ ]:
power_green_four.shape,laser_four.shape

In [ ]:
# fig = go.Figure()
# laser=fun.laser_scaling(data_green_new_3,power_green_new_3,np.array([528,536]))

# fig.add_scatter(x=fun.origin(power_green_new_3),y=fun.origin(laser),name='integrated laser PL')
# fig.add_scatter(x=fun.origin(power_green_four),y=fun.origin(laser_four),name='integrated laser_four PL')
# fig.add_scatter(x=fun.origin(power_green_new_3),y=fun.origin(scale_set3*laser),name='scaled Pl of Laser')
# fig.add_scatter(x=fun.origin(power_green_new_3),y=fun.origin(intensity_green_new_3),name='integrated 532nm PL from NV')
# fig.add_scatter(x=fun.origin(power_green_new_3),y=fun.origin(scale_set3*intensity_green_new_3),name='scaled 532nm PL of NV')
# fig.update_layout(xaxis = dict(title = 'Green Power (\N{greek small letter mu}W)',range=[0,90]),yaxis = dict(title = 'Integrated PL' ))
# fig.show()

In [ ]:
# fig = go.Figure()

# fig.add_scatter(x=power_green_new,y=intensity_green_new,name='Only Green New (Set 2)',mode="markers",marker=dict( size=8,color='MediumPurple',symbol='circle-open'),opacity=1.0,)
# fig.add_scatter(x=power_green_new_3,y=intensity_green_new_3,name='Only Green New_3 (Set 3)',mode="markers",marker=dict( size=6,color='royalblue',symbol='triangle-up'),opacity=0.9,)
# fig.add_scatter(x=power_green_new_50_20,y=intensity_green_new_50_20,name='Only Green New_50_20 (Set 3)',mode="markers",marker=dict( size=6,color='LightSkyBlue',symbol='diamond'),opacity=0.8,)
# fig.add_scatter(x=fun.origin(power_green_new_3),y=fun.origin(scale_set3*intensity_green_new_3),name='scaled 532nm PL of NV')
# # fig.add_scatter(x=power_green_new_2,y=intensity_green_new_2,name='Only Green New_2 (Set 3)(not used)',mode="markers",marker=dict( size=6,color='LightSkyBlue',symbol='star'),opacity=0.5,)

# #Set1
# # fig.add_scatter(x=greenOnly_powers,y=intensity_green,name='Only Green (Set 1)',opacity=0.5,)
# # for i in range(data_1064.shape[0]):
# #     fig.add_scatter(x=green_powers,y=intensity_1064[i,:],name=f'1064nm power : {ir_powers[i]} mW ')
# #     fig.add_scatter(x=green_powers,y=intensity_975[i,:],name=f'975nm power : {ir_powers[i]} mW ')

# #Set2
# for i in range(len(power_1064_new)):
#     scale_set_1064 = fun.interpolation(power_green_new_3,scale_set3, power_1064_new[i])
#     fig.add_scatter(x=fun.origin(power_1064_new[i]),y=fun.origin(intensity_1064_new[i]),name=f'1064nm power (Set 2) : {ir_powers_new[i]} mW ',mode="markers",)
#     fig.add_scatter(x=fun.origin(power_1064_new[i]),y=fun.origin(scale_set_1064*intensity_1064_new[i]),name=f'1064nm power (Set 2) scaled: {ir_powers_new[i]} mW ',)
#     scale_set_975 = fun.interpolation(power_green_new_3,scale_set3, power_975_new[i])
#     fig.add_scatter(x=fun.origin(power_975_new[i]),y=fun.origin(intensity_975_new[i]),name=f'975nm power (Set 2) : {ir_powers_new[i]} mW ',mode="markers",)
#     fig.add_scatter(x=fun.origin(power_975_new[i]),y=fun.origin(scale_set_975*intensity_975_new[i]),name=f'975nm power (Set 2) scaled: {ir_powers_new[i]} mW ')


# #Set2
# # for i in irs:
# #     for j in range(len(ir_powers_new)):
# #         globals()[f'scale_set_{i}_{ir_powers_new[j]}'] = fun.interpolation(power_green_new_3,scale_set3, globals()[f'power_{i}_new'][j])
# #         fig.add_scatter(x=fun.origin(globals()[f'power_{i}_new'][j]),y=fun.origin(globals()[f'intensity_{i}_new'][j]),name=f'{i}nm power (Set 2) : {ir_powers_new[j]} mW ',mode="markers",)
# #         fig.add_scatter(x=fun.origin(globals()[f'power_{i}_new'][j]),y=fun.origin(globals()[f'scale_set_{i}_{ir_powers_new[j]}']\
# #

# #Set3
# for i in range(len(power_975_new_2)):
#     scale_set_975_2 = fun.interpolation(power_green_new_3,scale_set3, power_975_new_2[i])
#     fig.add_scatter(x=fun.origin(power_975_new_2[i]),y=fun.origin(intensity_975_new_2[i]),name=f'975nm power (Set 3): {ir_powers_new_2[i]} mW ',mode="markers",)
#     fig.add_scatter(x=fun.origin(power_975_new_2[i]),y=fun.origin(scale_set_975_2*intensity_975_new_2[i]),name=f'975nm power Scaled (Set 3): {ir_powers_new_2[i]} mW ')

# fig.update_layout(xaxis = dict(title = 'Green Power (\N{greek small letter mu}W)',range=[0,90]),yaxis = dict(title = 'Integrated PL' ))
# fig

##### The applicable Plots

In [ ]:
#detector scaling on the second dataSets by 1064nm lowest power
scale_two=fun.detector_scaling(data_1064_new[0],power_1064_new[0],np.array([528,536]))
scale_two_green=fun.interpolation(power_1064_new[-1],scale_two,power_green_new)

#detector scaling on the fourth dataSets
scale_four=fun.detector_scaling(data_green_four,power_green_four,np.array([528,536]))

In [ ]:
fig = go.Figure()
laser_two=fun.laser_scaling(data_1064_new[0],power_1064_new[0],np.array([528,536]))
laser_four=fun.laser_scaling(data_green_four,power_green_four,np.array([528,536]))

fig.add_scatter(x=fun.origin(power_green_new),y=fun.origin(laser_two),name='integrated laser_two PL')
fig.add_scatter(x=fun.origin(power_green_four),y=fun.origin(laser_four),name='integrated laser_four PL')
fig.add_scatter(x=fun.origin(power_green_new),y=fun.origin(scale_two*laser_two),name='scaled Pl of Laser_two')
fig.add_scatter(x=fun.origin(power_green_four),y=fun.origin(scale_four*laser_four),name='scaled Pl of Laser_four')
fig.add_scatter(x=fun.origin(power_green_new),y=fun.origin(intensity_green_new),name='integrated 532nm PL from NV_secondSet')
fig.add_scatter(x=fun.origin(power_green_new),y=fun.origin(scale_two_green*intensity_green_new),name='scaled 532nm PL of NV_secondSet')
fig.add_scatter(x=fun.origin(power_green_four),y=fun.origin(intensity_green_four),name='integrated 532nm PL from NV_fourthSet')
fig.add_scatter(x=fun.origin(power_green_four),y=fun.origin(scale_four*intensity_green_four),name='scaled 532nm PL of NV_fourthSet')

fig.update_layout(template=fig_template,height=500, width=1300,xaxis = dict(title = 'Green Power (\N{greek small letter mu}W)',range=[0,90]),yaxis = dict(title = 'Integrated PL' ))
fig.show()

In [ ]:
ir_powers_new

In [ ]:
#green and 1064nm from the Second dataSets
fig = go.Figure()

# # #uncorrected Plots
# fig = make_subplots(rows=5, cols=1, shared_xaxes=True, vertical_spacing=0.10)
# for i in range(data_green_new.shape[1]-1):
#     fig.add_scatter(x=data_green_new[:,0],y=data_green_new[:,i+1],name=f'Green power : {np.round(power_green_new[i],3)} \N{greek small letter mu}W',row=1,col=1)
#     fig.update_xaxes(title_text='Wavelenght (nm)', row=1, col=1)
#     fig.update_yaxes(title_text='PL of 532nm (Second Set)', row=1, col=1)

# for j in range(len(ir_powers_new)):
#     for i in range(globals()['data_1064_'+f'{ir_powers_new[j]}'].shape[1]-1):
#         fig.add_scatter(x=globals()['data_1064_'+f'{ir_powers_new[j]}'][:,0],y=globals()['data_1064_'+f'{ir_powers_new[j]}'][:,i+1] ,
#                         name=f'IR_'+f'{ir_powers_new[j]}mW_Green : {np.round(power_1064_new[j][i],3)} \N{greek small letter mu}W',row=j+2, col=1)
#         fig.update_xaxes(title_text=f"Wavelenght", row=j+2, col=1)
#         fig.update_yaxes(title_text=f"PL_1064nm_{ir_powers_new[j]}mW", row=j+2, col=1)
# fig.update_layout(height=1300, width=1300,
#                   title_text="Second PL Set of 1064nm")

#corrected Plots
data_green_scaled=np.ones((data_green_new.shape))
data_green_scaled[:,0]=data_green_new[:,0]
data_green_scaled[:,1:]=data_green_new[:,1:]*scale_two_green

irs = ['1064']
for i in irs:
    globals()[f'data_{i}_new_scaled']=[]
    for j in range(len(ir_powers_new)):
        globals()[f'scale_set_{i}_{ir_powers_new[j]}'] = fun.interpolation(power_green_new,scale_two_green, globals()[f'power_{i}_new'][j])
        globals()[f'data_{i}_new_scaled_{ir_powers_new[j]}']=np.ones((globals()[f'data_{i}_new'][j].shape))
        for k in range(len(globals()[f'power_{i}_new'][j])):
            globals()[f'data_{i}_new_scaled_{ir_powers_new[j]}'][:,0]=globals()[f'data_{i}_new'][j][:,0]
            globals()[f'data_{i}_new_scaled_{ir_powers_new[j]}'][:,k+1]=(globals()[f'data_{i}_new'][j][:,k+1]*globals()[f'scale_set_{i}_{ir_powers_new[j]}'][k])   #like data_1064_new_scaled_100
            globals()[f'data_{i}_new_scaled'].append(globals()[f'data_{i}_new_scaled_{ir_powers_new[j]}'])

#calculating intensity and ratio from the DetectorScaled PLs for green and 1064nm
scaled_intensity_green,scaled_ratio_green = fun.intensity(data_green_scaled,power_green_new,cutOff,pl_range=pl_range,dim=2)

irs = ['1064']
for i in irs:
    globals()['intensity_'+f'{i}'+'_new_scaled']=[]
    globals()['ratio_'+f'{i}'+'_new_scaled']=[]
    for j in range(len(ir_powers_new)):
        globals()['intensity_'+f'{i}_'+f'{ir_powers_new[j]}_scaled'] = fun.intensity(globals()[f'data_{i}_new_scaled_{ir_powers_new[j]}'],globals()[f'power_{i}_{ir_powers_new[j]}'],cutOff,pl_range=pl_range,dim=2)[0]        #like : intensity_1064_100_scaled
        globals()['ratio_'+f'{i}_'+f'{ir_powers_new[j]}_scaled'] = fun.intensity(globals()[f'data_{i}_new_scaled_{ir_powers_new[j]}'],globals()[f'power_{i}_{ir_powers_new[j]}'],cutOff,pl_range=pl_range,dim=2)[1]            #like : ratio_1064_100_scaled
        print('ratio_'+f'{i}_'+f'{ir_powers_new[j]}_scaled')
        globals()['intensity_'+f'{i}'+'_new_scaled'].append(globals()['intensity_'+f'{i}_'+f'{ir_powers_new[j]}_scaled'])            #like intensity_1064_new_scaled
        globals()['ratio_'+f'{i}'+'_new_scaled'].append(globals()['ratio_'+f'{i}_'+f'{ir_powers_new[j]}_scaled'])                   #like ratio_1064_new_scaled  with mean of each ir_powers
    globals()['ratio_'+f'{i}'+'_new_scaled']=np.array(globals()['ratio_'+f'{i}'+'_new_scaled'])


ratio_green_scaled_new=np.mean(scaled_ratio_green)
ratio_1064_scaled_new=np.mean(ratio_1064_new_scaled)
print(f'Ratios from Detector_Scaled PLs: (532nm,{np.round(scaled_ratio_green,4)}) (532nm_average,{np.round(ratio_green_scaled_new,4)})  (1064nm,{np.round(ratio_1064_new_scaled,4)}) (1064nm_average,{np.round(ratio_1064_scaled_new,4)})\n')

fig = make_subplots(rows=5, cols=1, shared_xaxes=True, vertical_spacing=0.10)
for i in range(data_green_new.shape[1]-1):
    fig.add_scatter(x=data_green_new[:,0],y=data_green_scaled[:,i+1],name=f'corrected Green power_set2 : {np.round(power_green_new[i],3)} \N{greek small letter mu}W',row=1, col=1) #with Correction
    fig.update_xaxes(title_text='Wavelenght (nm)', row=1, col=1)
    fig.update_yaxes(title_text='PL_532nm (Second Set)', row=1, col=1)

irs = ['1064']
for i_ir in irs:
    for j in range(len(ir_powers_new)):
        for i in range(globals()[f'data_{i_ir}_new_scaled_'+f'{ir_powers_new[j]}'].shape[1]-1):
            #with correction
            fig.add_scatter(x=globals()[f'data_{i_ir}_new_scaled_'+f'{ir_powers_new[j]}'][:,0],y=globals()[f'data_{i_ir}_new_scaled_'+f'{ir_powers_new[j]}'][:,i+1] ,       #like data_1064_new_scaled_100
                            name=f'IR_'+f'{ir_powers_new[j]}mW_Green_corrected : {np.round(globals()[f"power_{i_ir}_new"][j][i],3)} \N{greek small letter mu}W',row=j+2, col=1)
            # # #without correction
            # fig.add_scatter(x=globals()[f'data_{i_ir}_'+f'{ir_powers_new[j]}'][:,0],y=globals()[f'data_{i_ir}_'+f'{ir_powers_new[j]}'][:,i+1] ,
            #                 name=f'IR_'+f'{ir_powers_new[j]}mW_Green : {np.round(globals()[f"power_{i_ir}_new"][j][i],3)} \N{greek small letter mu}W',row=j+2, col=1)
            fig.update_xaxes(title_text=f"Wavelenght", row=j+2, col=1)
            fig.update_yaxes(title_text=f"PL_{i_ir}nm_{ir_powers_new[j]}mW", row=j+2, col=1)
fig.update_layout(template=fig_template,height=1300, width=1300,
                  title_text=f"Detector Scaled Second PL Set of Green and {i_ir}nm")
# fig.write_html("Second PL Set.html")
fig.show()

In [ ]:
#green and 975nm from the fourth dataSets
fig = go.Figure()

# #uncorrected Plots
# fig = make_subplots(rows=5, cols=1, shared_xaxes=True, vertical_spacing=0.10)
# for i in range(data_green_four.shape[1]-1):
#     fig.add_scatter(x=data_green_four[:,0],y=data_green_four[:,i+1],name=f'Green power : {np.round(power_green_four[i],3)} \N{greek small letter mu}W', row=1, col=1)
#     fig.update_xaxes(title_text='Wavelenght (nm)', row=1, col=1)
#     fig.update_yaxes(title_text='PL of 532nm (Second Set)', row=1, col=1)


# for j in range(len(ir_powers_four)):
#     for i in range(globals()['data_975_'+f'{ir_powers_four[j]}'].shape[1]-1):
#         fig.add_scatter(x=globals()['data_975_'+f'{ir_powers_four[j]}'][:,0],y=globals()['data_975_'+f'{ir_powers_four[j]}'][:,i+1],
#                         name=f'IR_'+f'{ir_powers_four[j]}mW_Green : {np.round(power_975_four[j][i],3)} \N{greek small letter mu}W', row=j+2, col=1)
#         fig.update_xaxes(title_text=f"Wavelenght", row=j+2, col=1)
#         fig.update_yaxes(title_text=f"PL_975nm_{ir_powers_four[j]}mW", row=j+2, col=1)
# fig.update_layout(height=1300, width=1300,
#                   title_text="Second PL Set of 975nm")

#corrected plots
data_green_scaled_four=np.ones((data_green_four.shape))
data_green_scaled_four[:,0]=data_green_four[:,0]
data_green_scaled_four[:,1:]=data_green_four[:,1:]*scale_four

irs = ['975']
for i in irs:
    globals()[f'data_{i}_four_scaled']=[]
    for j in range(len(ir_powers_four)):
        globals()[f'scale_set_{i}_{ir_powers_four[j]}'] = fun.interpolation(power_green_four,scale_four, globals()[f'power_{i}_four'][j])
        globals()[f'data_{i}_four_scaled_{ir_powers_four[j]}']=np.ones((globals()[f'data_{i}_four'][j].shape))
        for k in range(len(globals()[f'power_{i}_four'][j])):
            globals()[f'data_{i}_four_scaled_{ir_powers_four[j]}'][:,0]=globals()[f'data_{i}_four'][j][:,0]
            globals()[f'data_{i}_four_scaled_{ir_powers_four[j]}'][:,k+1]=(globals()[f'data_{i}_four'][j][:,k+1]*globals()[f'scale_set_{i}_{ir_powers_four[j]}'][k])   #like data_975_four_scaled_100
            globals()[f'data_{i}_four_scaled'].append(globals()[f'data_{i}_four_scaled_{ir_powers_four[j]}'])                                                          #like data_975_four_scaled


##Calculating Intensity and Ratios from the DetectorScaled PLs for green and 975nm
scaled_intensity_green_four,scaled_ratio_green_four = fun.intensity(data_green_scaled_four,power_green_four,cutOff,pl_range=pl_range,dim=2)

irs = ['975']
for i in irs:
    globals()['intensity_'+f'{i}'+'_four_scaled']=[]
    globals()['ratio_'+f'{i}'+'_four_scaled']=[]
    for j in range(len(ir_powers_four)):
        globals()['intensity_'+f'{i}_'+f'{ir_powers_four[j]}_scaled'] = fun.intensity(globals()[f'data_{i}_four_scaled_{ir_powers_four[j]}'],globals()[f'power_{i}_{ir_powers_four[j]}'],cutOff,pl_range=pl_range,dim=2)[0]        #like : intensity_975_100_scaled
        globals()['ratio_'+f'{i}_'+f'{ir_powers_four[j]}_scaled'] = fun.intensity(globals()[f'data_{i}_four_scaled_{ir_powers_four[j]}'],globals()[f'power_{i}_{ir_powers_four[j]}'],cutOff,pl_range=pl_range,dim=2)[1]            #like : ratio_975_100_scaled
        print('ratio_'+f'{i}_'+f'{ir_powers_four[j]}_scaled')
        globals()['intensity_'+f'{i}'+'_four_scaled'].append(globals()['intensity_'+f'{i}_'+f'{ir_powers_four[j]}_scaled'])            #like intensity_975_four_scaled
        globals()['ratio_'+f'{i}'+'_four_scaled'].append(globals()['ratio_'+f'{i}_'+f'{ir_powers_four[j]}_scaled'])                    #like ratio_975_four_scaled  with mean of each ir_powers
    globals()['ratio_'+f'{i}'+'_four_scaled']=np.array(globals()['ratio_'+f'{i}'+'_four_scaled'])


ratio_green_scaled_four=np.mean(scaled_ratio_green_four)
ratio_975_scaled_four=np.mean(ratio_975_four_scaled)
ratio_green_unscaled_four=np.mean([ratio_green_four])
ratio_975_unscaled_four=np.mean(ratio_975_four)
print(f'Ratios from Detector_unScaled PLs: (532nm,{np.round(scaled_ratio_green_four,4)}) (532nm_average,{np.round(ratio_green_unscaled_four,4)})  \n(975nm,{np.round(ratio_975_four,4)}) (975nm_average,{np.round(ratio_975_unscaled_four,4)})\n')
print(f'Ratios from Detector_Scaled PLs: (532nm,{np.round(ratio_green_scaled_four,4)})  \n(975nm,{np.round(ratio_975_four_scaled,4)}) (975nm_average,{np.round(ratio_975_scaled_four,4)})\n')

fig = make_subplots(rows=5, cols=1, shared_xaxes=True, vertical_spacing=0.10)
for i in range(data_green_four.shape[1]-1):
    fig.add_scatter(x=data_green_four[:,0],y=data_green_scaled_four[:,i+1],name=f'corrected Green power_set4 : {np.round(power_green_four[i],3)} \N{greek small letter mu}W', row=1, col=1) #with Correction
    fig.update_xaxes(title_text='Wavelenght (nm)', row=1, col=1)
    fig.update_yaxes(title_text='PL_532nm (Second Set)', row=1, col=1)

irs = ['975']
for i_ir in irs:
    for j in range(len(ir_powers_four)):
        for i in range(globals()[f'data_{i_ir}_four_scaled_'+f'{ir_powers_four[j]}'].shape[1]-1):
            #with correction
            fig.add_scatter(x=globals()[f'data_{i_ir}_four_scaled_'+f'{ir_powers_four[j]}'][:,0],y=globals()[f'data_{i_ir}_four_scaled_'+f'{ir_powers_four[j]}'][:,i+1] ,       #like data_975_four_scaled_100
                            name=f'IR_'+f'{ir_powers_four[j]}mW_Green_corrected : {np.round(globals()[f"power_{i_ir}_four"][j][i],3)} \N{greek small letter mu}W',row=j+2, col=1)
            # # #without correction
            # fig.add_scatter(x=globals()[f'data_{i_ir}_'+f'{ir_powers_four[j]}'][:,0],y=globals()[f'data_{i_ir}_'+f'{ir_powers_four[j]}'][:,i+1] ,
            #                 name=f'IR_'+f'{ir_powers_four[j]}mW_Green : {np.round(globals()[f"power_{i_ir}_four"][j][i],3)} \N{greek small letter mu}W',row=j+2, col=1)
            fig.update_xaxes(title_text=f"Wavelenght", row=j+2, col=1)
            fig.update_yaxes(title_text=f"PL_{i_ir}nm_{ir_powers_four[j]}mW", row=j+2, col=1)
fig.update_layout(template=fig_template,height=1300, width=1300,
                  title_text=f"Detector Scaled Fourth PL Set of Green and {i_ir}nm")

# fig.write_html("Fourth PL Set.html")
fig.show()

In [ ]:
np.mean(np.round(ratio_975_four,4),axis=1)

In [ ]:
np.mean(np.round(ratio_1064_new_scaled,4),axis=1)

In [ ]:
# # #green scaling to match both the references
scale_four_two = fun.green_scale(scaled_intensity_green,scaled_intensity_green_four,power_green_new,power_green_four)

# rescaled_intensity_green_four=scale_four_two*scaled_intensity_green_four
# intensity_975_four_rescaled=np.ones((np.shape(intensity_975_four_scaled)))
# for i in range(len(intensity_975_four_scaled)):
#     intensity_975_four_rescaled[i]=scale_four_two*intensity_975_four_scaled[i]

In [ ]:
# # #green scaling to match both the references
green_scale_four_two=scaled_intensity_green/scaled_intensity_green_four

rescaled_intensity_green_four=green_scale_four_two*scaled_intensity_green_four
intensity_975_four_rescaled=np.ones((np.shape(intensity_975_four_scaled)))
for i in range(len(intensity_975_four_scaled)):
    intensity_975_four_rescaled[i]=green_scale_four_two*intensity_975_four_scaled[i]

In [ ]:
green_scale_four_two,scale_four_two

In [ ]:
green4_ref_correct = scaled_intensity_green/rescaled_intensity_green_four

In [ ]:
green4_ref_correct,green_scale_four_two

In [ ]:
fig = go.Figure()

overall_green_intensity=(scaled_intensity_green+rescaled_intensity_green_four)/2.0

fig.add_scatter(x=fun.origin(power_green_new),y=fun.origin(scaled_intensity_green),name='Only Green Two',)
# fig.add_scatter(x=fun.origin(power_green_new_3),y=fun.origin(scale_set3*intensity_green_new_3),name='Only Green Three')
fig.add_scatter(x=fun.origin(power_green_four),y=fun.origin(rescaled_intensity_green_four),name='Only Green four',)
# fig.add_scatter(x=fun.origin(power_green_four),y=fun.origin(overall_green_intensity),name='overall_green_intensity',)
for i in range(len(power_975_four)):
    # fig.add_scatter(x=fun.origin(power_1064_new[i]),y=fun.origin(intensity_1064_new[i]),name=f'1064nm power two : {ir_powers_new [i]} mW ',)
    fig.add_scatter(x=fun.origin(power_975_four[i]),y=fun.origin(green4_ref_correct*intensity_975_four_rescaled[i]),name=f'975nm power four : {ir_powers_four [i]} mW ',)
    # scale_set_975_2 = fun.interpolation(power_green_new_3,scale_set3, power_975_new_2[i])
    # fig.add_scatter(x=fun.origin(power_975_new_2[i]),y=fun.origin(scale_set_975_2*intensity_975_new_2[i]),name=f'975nm power Three: {ir_powers_new_2[i]} mW ')
fig.update_layout(xaxis = dict(title = 'Green Power (\N{greek small letter mu}W)'),yaxis = dict(title = 'Integrated PL' ))
fig.update_layout(template=fig_template,height=500, width=1300,
                  title_text=f"Integrated PL Set of Green and IRs")

# fig.write_html("IntegratedPL_GreenPower.html")
fig.show()

### The <span style= 'color: Green'> $\textbf{Modeling}$ </span>

#### The Evolution matrix

In [ ]:
r1,r1_p,r2,r2_p,alpha1,alpha2,k1,k2,beta1,beta2,l1,l2,gamma1,gamma2,delta,I_g,IR_1064,IR_975= sympy.symbols('r1 r1_p r2 r2_p alpha1 alpha2 k1 k2 beta1 beta2 l1 l2 gamma1 gamma2 delta I_g IR_1064 IR_975')
sympy_matrix = sympy.Matrix([[(-(r1*I_g)-(k1*IR_1064*IR_1064)-(l1*IR_975*IR_975)-(delta*IR_975*IR_975)),r1_p,0,(alpha2*I_g+beta2*IR_1064+gamma2*IR_975),0],[(r1*I_g+(k1*IR_1064*IR_1064)+(l1*IR_975*IR_975)),(-alpha1*I_g-r1_p-beta1*IR_1064-gamma1*IR_975),0,0,0],[delta*IR_975*IR_975,(alpha1*I_g+beta1*IR_1064+gamma1*IR_975),(-r2*I_g-(k2*IR_1064*IR_1064)-(l2*IR_975*IR_975)),r2_p,0],[0,0,(r2*I_g+(k2*IR_1064*IR_1064)+(l2*IR_975*IR_975)),(-alpha2*I_g-r2_p-beta2*IR_1064-gamma2*IR_975),0],[1,1,1,1,-1]])
sympy_matrix

#### Finding the States

In [ ]:
print(f'Ratios from Detector_Scaled PLs: (532nm,{np.round(ratio_green_scaled_new,4)})  (975nm,{np.round(ratio_975_scaled_four,4)})  (1064nm,{np.round(ratio_1064_scaled_new,4)})\n')

In [ ]:
2.1632*0.000280/1.27

In [ ]:
#To calculate the states for green
power_g = np.linspace(40,90,10)
# power_g=power_green_new
r1_r1p=np.linspace(0.002,0.002,1)
alpha1=np.linspace(0.007,0.007,1)
# alpha1=500*np.linspace(0.0065,0.0065,1)
# alpha2_alpha1 = np.linspace(scaled_ratio_green[0],scaled_ratio_green[0],1)
alpha2_alpha1 = np.linspace(2.1632,2.1632,1)

laser_green = np.ones((len(r1_r1p),len(alpha1),len(alpha2_alpha1),len(power_g),5))
ionization_green =np.ones((len(r1_r1p),len(alpha1),len(alpha2_alpha1),len(power_g)))
recombination_green2=np.ones((len(r1_r1p),len(alpha1),len(alpha2_alpha1),len(power_g)))
ionization_green2 =np.ones((len(r1_r1p),len(alpha1),len(alpha2_alpha1),len(power_g)))
recombination_green=np.ones((len(r1_r1p),len(alpha1),len(alpha2_alpha1),len(power_g)))
rec_ion_ratio_green=np.ones((len(r1_r1p),len(alpha1),len(alpha2_alpha1),len(power_g)))
rec_ion_ratio_green2=np.ones((len(r1_r1p),len(alpha1),len(alpha2_alpha1),len(power_g)))

chi_green = np.ones((len(r1_r1p),len(alpha2_alpha1)))

guess = np.array([1,0,0,0,0])
check = np.zeros(5,dtype= float)

print(f'Power\t r1_ratio\t alpha1\t alpha_ratio\t ionization\t rec_ion_ratio\t g1\t\t e1\t\t g2\t\t e2')
#loop for only green
for i_alpha1 in range(len(alpha1)):
    for i_r1_r1p in range(len(r1_r1p)):
        for i_alpha in range(len(alpha2_alpha1)):
            ir = 0
            for i in range(len(power_g)):
                    r2_r2p=r1_r1p[i_r1_r1p]*(21.0/12.0)
                    coff = np.array([r1_r1p[i_r1_r1p],r2_r2p,alpha1[i_alpha1],alpha2_alpha1[i_alpha]])    #### first Knob
                    power = np.array([power_g[i],0,0])
                    laser_green[i_r1_r1p,i_alpha1,i_alpha,i,:] = fsolve(fun.state_green,guess,args = (power,coff))

                    nr1,nr1_p,nr2,nr2_p,nalpha1,nalpha2=fun.cof(*power,coff)[0:6]
                    ionization_green[i_r1_r1p,i_alpha1,i_alpha,i]=fun.ion(power_g[i],laser_green[i_r1_r1p,i_alpha1,i_alpha,i,0],laser_green[i_r1_r1p,i_alpha1,i_alpha,i,1],nr1,nr1_p,nalpha1)
                    recombination_green[i_r1_r1p,i_alpha1,i_alpha,i]=fun.rec(power_g[i],laser_green[i_r1_r1p,i_alpha1,i_alpha,i,2],laser_green[i_r1_r1p,i_alpha1,i_alpha,i,3],nr2,nr2_p,nalpha2)
                    ionization_green2[i_r1_r1p,i_alpha1,i_alpha,i]=fun.ion2(power_g[i],laser_green[i_r1_r1p,i_alpha1,i_alpha,i,0],laser_green[i_r1_r1p,i_alpha1,i_alpha,i,1],nr1,nr1_p,nalpha1)
                    recombination_green2[i_r1_r1p,i_alpha1,i_alpha,i]=fun.rec2(power_g[i],laser_green[i_r1_r1p,i_alpha1,i_alpha,i,2],laser_green[i_r1_r1p,i_alpha1,i_alpha,i,3],nr2,nr2_p,nalpha2)
                    rec_ion_ratio_green[i_r1_r1p,i_alpha1,i_alpha,i]=recombination_green[i_r1_r1p,i_alpha1,i_alpha,i]/ionization_green[i_r1_r1p,i_alpha1,i_alpha,i]
                    rec_ion_ratio_green2[i_r1_r1p,i_alpha1,i_alpha,i]=recombination_green2[i_r1_r1p,i_alpha1,i_alpha,i]/ionization_green2[i_r1_r1p,i_alpha1,i_alpha,i]

                    print(f'{np.round(power_g[i],3)}\t {np.round(r1_r1p[i_r1_r1p],3)}\t {np.round(alpha1[i_alpha1],4)}\t{np.round(alpha2_alpha1[i_alpha],3)}\t'+\
                          f'\t {np.round(ionization_green[i_r1_r1p,i_alpha1,i_alpha,i],7)}\t {np.round(rec_ion_ratio_green[i_r1_r1p,i_alpha1,i_alpha,i],7)}\t\t'\
                         +f'{np.round(laser_green[i_r1_r1p,i_alpha1,i_alpha,i,0],7)}\t{np.round(laser_green[i_r1_r1p,i_alpha1,i_alpha,i,1],7)}\t{np.round(laser_green[i_r1_r1p,i_alpha1,i_alpha,i,2],7)}\t{np.round(laser_green[i_r1_r1p,i_alpha1,i_alpha,i,3],7)}')

print(f'\nIon_Rec rate from Simulation : { rec_ion_ratio_green}\n Recombination : {recombination_green}')
pl_green = (np.squeeze(laser_green)[:,1]*1e3*(1/12)+np.squeeze(laser_green)[:,3]*1e3*(1/12)*(12.0/21.0))
ratio_green_simul = (np.squeeze(laser_green)[:,1]*1e3*(1/12))/(np.squeeze(laser_green)[:,3]*1e3*(1/12)*(12.0/21.0))
print(f'\nRatios from Exp : {np.mean(scaled_ratio_green)}\nfrom Simulation : { ratio_green_simul}')
                    # print(np.sum(laser_green[i_alpha,g,0:-1]))
                    # verification = np.isclose(fun.state_green(laser_green[i_alpha,g,:],*(power,coff)), check)  # verification of the roots
                    # print(verification)
# chi_green[i_alpha] = fun.chi_sqr(fun.organize(intensity_green,'i',2),fun.organize(laser_green[i_alpha,:,1] + laser_green[i_alpha,:,3],'j',2))

In [ ]:
e1_norm = np.squeeze(laser_green)[:,1]/(np.squeeze(laser_green)[:,1]+np.squeeze(laser_green)[:,0])

In [ ]:
fig = go.Figure()
fig.add_scatter(x=power_g,y=e1_norm,mode="markers",name='Normalized excited state population')
fig.add_scatter(x=power_g,y=power_g*0.074/40,mode="lines",)
fig.add_scatter(x=power_g,y=np.squeeze(laser_green)[:,1],mode="markers",)
fig.add_scatter(x=power_g,y=(np.squeeze(laser_green)[:,1]+np.squeeze(laser_green)[:,0]),mode="markers",)
fig.add_scatter(x=power_g,y=(np.squeeze(laser_green)[:,2]+np.squeeze(laser_green)[:,3]),mode="markers",)
fig.add_scatter(x=power_g,y=(np.squeeze(laser_green)[:,1]+np.squeeze(laser_green)[:,0])/(np.squeeze(laser_green)[:,2]+np.squeeze(laser_green)[:,3]),mode="markers",)
fig.add_scatter(x=power_g,y=(np.squeeze(laser_green)[:,1])/(np.squeeze(laser_green)[:,3]),mode="markers",)
# fig.add_scatter(x=power_g,y=power_g*0.219/40,mode="lines",)

fig.update_layout(height=500,width=500)

In [ ]:
25*1e-6*power_g**2,60*1e-6*power_g**2

In [ ]:
scaled_ratio_green

In [ ]:
pl_factor = np.mean(scaled_intensity_green)/(np.mean(pl_green))

In [ ]:
fig = go.Figure()
pl_green_1 = pl_green*pl_factor
fig.add_scatter(x=fun.origin(power_g),y=fun.origin(pl_green_1),mode="lines",name='Simulated: 532nm')
fig.add_scatter(x=fun.origin(power_green_new),y=fun.origin(scaled_intensity_green),mode="markers",name='Only Green Two',)
fig.update_layout(xaxis = dict(title = 'Green Powers (\N{greek small letter mu}W)'),yaxis = dict(title = 'Integrated PL'))

In [ ]:
#taking the maximum power(100mW) to get the ratio as here 1064nm is prominent and the decreasing trend is also most apparant
ratio_1064_new_scaled

In [ ]:
2.2*1.4

In [ ]:
power_g

In [ ]:
1/1.27

In [ ]:
# #loop for green + 1064 IR
power_ir = ir_powers_new*1000     #experimental set of Powers
# power_ir = np.array([0.000001,*ir_powers_new])     #to cross_check the ionization and ratio with only green
# estimated_beta1=(1/((650*alpha1-alpha1)/alpha1))[0]     #equivalent to : # alpha1+beta1=number*alpha1
# r1_k1=np.array([np.inf]); alpha1_beta1=np.array([2.2])
r1_k1=np.array([np.inf]); alpha1_beta1=np.array([2.5])


ionization_1064 =np.ones(((len(r1_k1),len(alpha1_beta1),len(r1_r1p),len(alpha1),len(alpha2_alpha1),len(power_g),len(power_ir))))
recombination_1064=np.ones(((len(r1_k1),len(alpha1_beta1),len(r1_r1p),len(alpha1),len(alpha2_alpha1),len(power_g),len(power_ir))))
rec_ion_ratio_1064=np.ones(((len(r1_k1),len(alpha1_beta1),len(r1_r1p),len(alpha1),len(alpha2_alpha1),len(power_g),len(power_ir))))

laser_1064 = np.ones((len(r1_k1),len(alpha1_beta1),len(power_g),len(power_ir),5))
# chi_1064 = np.ones((len(alpha),len(k),len(beta)))

for i_k in range(len(r1_k1)):
    for i_beta in range(len(alpha1_beta1)):
        for i in range(len(power_g)):
            for j in range(len(power_ir)):
                r2_r2p=r1_r1p[0]*(21.0/12.0); r2_k2=r1_k1[i_k];alpha2_beta2=1.35*alpha1_beta1[i_beta]
                coff = np.array([r1_r1p[0],r2_r2p,alpha1[0],alpha2_alpha1[0],r1_k1[i_k],r2_k2,alpha1_beta1[i_beta],alpha2_beta2,ratio_1064_new_scaled[-1][0]])  #r1_r1p,r2_r2p,alpha1,alpha2_alpha1,r1_k1,r2_k2,alpha1_beta1,alpha2_beta2,charge_ratio
                power = np.array([power_g[i],0,power_ir[j]])
                laser_1064[i_k,i_beta,i,j,:] = fsolve(fun.state_green,guess,args = (power,coff))      #power_g,g1,e1,r1,r1p,alpha1,power_1064,k1,beta1

                nr1,nr1_p,nr2,nr2_p,nalpha1,nalpha2,nk1,nk2,nbeta1,nbeta2=fun.cof(*power,coff)[0:10]
                ionization_1064[i_k,i_beta,i_r1_r1p,i_alpha1,i_alpha,i,j]=fun.ion_1064(power_g[i],laser_1064[i_k,i_beta,i,j,0],laser_1064[i_k,i_beta,i,j,1],nr1,nr1_p,nalpha1,power_ir[j],nk1,nbeta1)
                recombination_1064[i_k,i_beta,i_r1_r1p,i_alpha1,i_alpha,i,j]=fun.rec_1064(power_g[i],laser_1064[i_k,i_beta,i,j,2],laser_1064[i_k,i_beta,i,j,3],nr2,nr2_p,nalpha2,power_ir[j],nk2,nbeta2)
                rec_ion_ratio_1064[i_k,i_beta,i_r1_r1p,i_alpha1,i_alpha,i,j]=recombination_1064[i_k,i_beta,i_r1_r1p,i_alpha1,i_alpha,i,j]/ionization_1064[i_k,i_beta,i_r1_r1p,i_alpha1,i_alpha,i,j]

# print(f'\nIon_Rec rate from Simulation : { rec_ion_ratio_1064} (for only 20mW, it is decreasing)')
pl_1064 = (np.squeeze(laser_1064)[:,:,1]*1e3*(1/12))+(np.squeeze(laser_1064)[:,:,3]*1e3*(1/12)*(12.0/21.0))
ratio_1064_simul=(np.squeeze(laser_1064)[:,:,1]*1e3*(1/12))/(np.squeeze(laser_1064)[:,:,3]*1e3*(1/12)*(12.0/21.0))
print(f'\nRatios from Exp : {np.mean(ratio_1064_new_scaled.T,axis=0)}\nfrom Simulation : {np.mean(ratio_1064_simul,axis=0)}')

In [ ]:
fig = go.Figure()
# fig.add_scatter(x=fun.origin(power_g),y=fun.origin(pl_green_1),mode="lines",name='Simulated: 532nm')
# fig.add_scatter(x=fun.origin(power_green_new),y=fun.origin(scaled_intensity_green),mode="markers",name='Only Green Two',)
pl_1064_1=np.ones((pl_1064.shape))

#Set2
for i in range(len(power_1064_new)):
    # scale_set_1064 = fun.interpolation(power_green_new,scale_two, power_1064_new[i])
    fig.add_scatter(x=fun.origin(power_1064_new[i]),y=fun.origin(intensity_1064_new_scaled[i]),name=f'Exp. Scaled 1064nm power : {power_ir[i]*1e-3} mW ',mode="markers",line=dict(width=2,color=px.colors.qualitative.Vivid[i]))

    pl_1064_1[:,i] = pl_1064[:,i]*pl_factor
    fig.add_scatter(x=fun.origin(power_g),y=fun.origin(pl_1064_1[:,i]),name=f'Simulated 1064nm power : {ir_powers_new[i]} mW ',mode="lines",line=dict(width=2,color=px.colors.qualitative.Vivid[i]))

fig.update_layout(xaxis = dict(title = 'Green Power (\N{greek small letter mu}W)',range=[0,90]),yaxis = dict(title = 'Integrated PL' ))
fig

In [ ]:
# fun.chi_sqr(simul=pl_1064_1,exper=(np.array(intensity_1064_new_scaled).T))/1e5

In [ ]:
numer = alpha1+(1/alpha1_beta1)*alpha1

In [ ]:
denom = alpha2_alpha1*alpha1+(1/alpha2_beta2)*(alpha2_alpha1*alpha1)

In [ ]:
1/alpha2_alpha1

In [ ]:
numer/denom

In [ ]:
0.005099-0.000596,0.003182-0.00035

In [ ]:
# #loop for green + 975 IR
power_ir = ir_powers_four*1000     #experimental set of Powers
# power_ir = np.array([0.000001,*ir_powers_new])     #to cross_check the ratio with only green
# estimated_gamma1=(1/((200*alpha1-alpha1)/alpha1))[0]     #equivalent to : # alpha1+gamma1=number*alpha1
r1_l1=np.array([np.inf]); r1_delta=np.array([np.inf]); alpha1_gamma1=np.array([15])

laser_975 = np.ones((len(r1_l1),len(alpha1_gamma1),len(r1_delta),len(power_g),len(power_ir),5))

ionization_975 =np.ones(((len(r1_l1),len(alpha1_gamma1),len(r1_delta),len(r1_r1p),len(alpha1),len(alpha2_alpha1),len(power_g),len(power_ir))))
recombination_975=np.ones(((len(r1_l1),len(alpha1_gamma1),len(r1_delta),len(r1_r1p),len(alpha1),len(alpha2_alpha1),len(power_g),len(power_ir))))
rec_ion_ratio_975=np.ones(((len(r1_l1),len(alpha1_gamma1),len(r1_delta),len(r1_r1p),len(alpha1),len(alpha2_alpha1),len(power_g),len(power_ir))))

for i_l in range(len(r1_l1)):
    for i_gamma in range(len(alpha1_gamma1)):
        for i_delta in range(len(r1_delta)):
            for i in range(len(power_g)):
                for j in range(len(power_ir)):

                    r2_r2p=r1_r1p[0]*(21.0/12.0); r2_l2=r1_l1[i_l];alpha2_gamma2=1.32*alpha1_gamma1[i_gamma]   ## previous finalized ratio 1.2
                    coff = np.array([r1_r1p[0],r2_r2p,alpha1[0],alpha2_alpha1[0],r1_l1[i_l],r2_l2,alpha1_gamma1[i_gamma],alpha2_gamma2,r1_delta[i_delta],0])  #r1_r1p,r2_r2p,alpha1,alpha2_alpha1,r1_l1,r2_l2,alpha1_gamma1,alpha2_gamma2,r1_delta,ratio_975_scaled
                    power = np.array([power_g[i],power_ir[j],0])
                    laser_975[i_l,i_gamma,i_delta,i,j,:] = fsolve(fun.state_green,guess,args = (power,coff))

                    nr1,nr1_p,nr2,nr2_p,nalpha1,nalpha2=fun.cof(*power,coff)[0:6]; nl1,nl2,ngamma1,ngamma2,ndelta=fun.cof(*power,coff)[-5:]
                    ionization_975[i_l,i_gamma,i_delta,i_r1_r1p,i_alpha1,i_alpha,i,j]=fun.ion_975(power_g[i],laser_975[i_l,i_gamma,i_delta,i,j,0],laser_975[i_l,i_gamma,i_delta,i,j,1],nr1,nr1_p,nalpha1,power_ir[j],nl1,ngamma1,ndelta)
                    recombination_975[i_l,i_gamma,i_delta,i_r1_r1p,i_alpha1,i_alpha,i,j]=fun.rec_975(power_g[i],laser_975[i_l,i_gamma,i_delta,i,j,2],laser_975[i_l,i_gamma,i_delta,i,j,3],nr2,nr2_p,nalpha2,power_ir[j],nl2,ngamma2,ndelta)
                    rec_ion_ratio_975[i_l,i_gamma,i_delta,i_r1_r1p,i_alpha1,i_alpha,i,j]=recombination_975[i_l,i_gamma,i_delta,i_r1_r1p,i_alpha1,i_alpha,i,j]/ionization_975[i_l,i_gamma,i_delta,i_r1_r1p,i_alpha1,i_alpha,i,j]

# print(f'\nIon_Rec rate from Simulation : { rec_ion_ratio_975} (all of them are increasing)')
pl_975 = (np.squeeze(laser_975)[:,:,1]*1e3*(1/12))+(np.squeeze(laser_975)[:,:,3]*1e3*(1/12)*(12.0/21.0))
ratio_975_simul=(np.squeeze(laser_975)[:,:,1]*1e3*(1/12))/(np.squeeze(laser_975)[:,:,3]*1e3*(1/12)*(12.0/21.0))
print(f'\nRatios from Exp : {np.mean(np.array(ratio_975_four).T,axis=0)}\nfrom Simulation : {np.mean(ratio_975_simul,axis=0)}')

    # chi_975[i_alpha,i_k,i_beta] = fun.chi_sqr(fun.organize(intensity_975,'i',2),fun.organize(laser_975[i_alpha,i_k,i_beta,:,:,1] + laser_975[i_alpha,i_k,i_beta,:,:,3],'j',2))

In [ ]:
np.mean(np.mean(ratio_975_simul,axis=0)),np.mean(np.mean(np.array(ratio_975_four).T,axis=0))

In [ ]:
2.1632*0.000466/1.32

In [ ]:
fig = go.Figure()
# fig.add_scatter(x=fun.origin(power_g),y=fun.origin(pl_green_1),mode="lines",name='Simulated: 532nm')
# fig.add_scatter(x=fun.origin(power_green_new),y=fun.origin(scaled_intensity_green),mode="markers",name='Only Green Two',)
pl_975_1=np.ones((pl_975.shape))

#Set2
for i in range(len(power_975_four)):
    # scale_set_975 = fun.interpolation(power_green_new,scale_two, power_975_four[i])
    # fig.add_scatter(x=fun.origin(power_975_four[i]),y=fun.origin(intensity_975_four_scaled[i]),name=f'Exp. Scaled 975nm power : {power_ir[i]} mW ',mode="markers",line=dict(width=3,color=px.colors.qualitative.Vivid[i]))
    fig.add_scatter(x=fun.origin(power_975_four[i]),y=fun.origin(green4_ref_correct*intensity_975_four_rescaled[i]),name=f'975nm power : {power_ir [i]*1e-3} mW ',mode='markers',line=dict(width=2,color=px.colors.qualitative.Vivid[i]))

    pl_975_1[:,i] = pl_975[:,i]*pl_factor
    fig.add_scatter(x=fun.origin(power_g),y=fun.origin(pl_975_1[:,i]),name=f'Simulated 975nm power : {ir_powers_new[i]} mW ',mode="lines",line=dict(width=2,color=px.colors.qualitative.Vivid[i]))

fig.update_layout(xaxis = dict(title = 'Green Power (\N{greek small letter mu}W)',range=[40,90]),yaxis = dict(title = 'Integrated PL' ))
fig

In [ ]:
print(f'532nm ratios : \nSimul {ratio_green_simul}, \nExp. {scaled_ratio_green}\n')
print('Columns indicate for the IR powers from 20mW to 100mW,\nRows indicate for the 532nm powers in increasing order\n')
print(f'1064nm ratios : \nSimul {ratio_1064_simul}, \n\nExp. {ratio_1064_new_scaled.T}\n')
print(f'975nm ratios : \nSimul {ratio_975_simul}, \n\nExp. {np.array(ratio_975_four).T}')

In [ ]:
print(f'\nRecombination Rate/Ionization Rate, \n\nGreen :{ rec_ion_ratio_green} \n1064nm :{ rec_ion_ratio_1064} \n975nm :{ rec_ion_ratio_975}\n\n(Comments:  \n1.for only 1064nm 20mW, it is decreasing\n2.for 20mW and 100mW, the rate is less for 975nm than 1064nm)')

In [ ]:
# fig = go.Figure()
# # fig.add_scatter(x=fun.origin(power_g),y=fun.origin(pl_green_1),name='Simulated: 532nm')
# # fig.add_scatter(x=fun.origin(power_green_new_3),y=fun.origin(scale_set3*intensity_green_new_3),name='Scaled Exp: 532nm',mode='markers',)
# pl_975_2=np.ones((pl_975.shape));pl_975_3=np.ones((pl_975.shape))

# ##Set2 : for 20 and 50mW
# for i in range(len(power_975_new)-2):
#     scale_set_975 = fun.interpolation(power_green_new_3,scale_set3, power_975_new[i])
#     fig.add_scatter(x=fun.origin(power_975_new[i]),y=fun.origin(scale_set_975*intensity_975_new[i]),name=f'Exp. Scaled 975nm power : {ir_powers_new[i]} mW ',mode="markers",)

#     pl_975_2[:,i] = pl_975[:,i]*pl_factor
#     fig.add_scatter(x=fun.origin(power_975_new[i]),y=fun.origin(pl_975_2[:,i]),name=f'Simulated 975nm power : {ir_powers_new[i]} mW ')

# ##Set3 : for 70 and 100mW
# for i in range(len(power_975_new_2)-2):
#     i+=2
#     scale_set_975_2 = fun.interpolation(power_green_new_3,scale_set3, power_975_new_2[i])
#     fig.add_scatter(x=fun.origin(power_975_new_2[i]),y=fun.origin(scale_set_975_2*intensity_975_new_2[i]),name=f'Exp. Scaled 975nm power : {ir_powers_new[i]} mW ',mode="markers",)

#     pl_975_3[:,i] = pl_975[:,i]*pl_factor
#     fig.add_scatter(x=fun.origin(power_975_new_2[i]),y=fun.origin(pl_975_3[:,i]),name=f'Simulated 975nm power : {ir_powers_new[i]} mW ')

# fig.update_layout(xaxis = dict(title = 'Green Power (\N{greek small letter mu}W)',range=[0,90]),yaxis = dict(title = 'Integrated PL' ))
# fig

In [ ]:
# laser_975 = np.ones((len(alpha),len(l),len(gamma),len(delta),len(power_g),len(power_ir),5))
# laser_1064 = np.ones((len(alpha),len(k),len(beta),len(power_g),len(power_ir),5))

# chi_975 = np.ones((len(alpha),len(l),len(gamma),len(delta)))
# chi_1064 = np.ones((len(alpha),len(k),len(beta)))

# # chi_1064 = fun.chi_sqr(fun.organize(intensity_1064,'i',2),fun.organize(states(power_g,power_ir)[2][:,:,1] +states(power_g,power_ir)[2][:,:,3],'j',2))
# # chi_975 = fun.chi_sqr(fun.organize(intensity_975,'i',2),fun.organize(states(power_g,power_ir)[1][:,:,1] +states(power_g,power_ir)[1][:,:,3],'j',2))

In [ ]:
#specifications for integral paralization
loops_number=2
cpu_count=joblib.cpu_count()
n_jobs=int(cpu_count/loops_number)

In [ ]:
def fsolve_state(guess,coff,power):
        laser_green = fsolve(fun.state_green,guess,args = (power,coff))
        return laser_green

In [ ]:
# %%time
# with joblib.parallel_backend(backend="loky"):
#     parallel = Parallel(n_jobs=n_jobs,verbose=False)
#     laser_green_1= parallel([delayed(fsolve_state)(guess,
#                                                    coff=np.array([r1_r1p,r2_r2p,alpha[i_alpha]]),
#                                                    power=np.array([power_g[i],0,0])) for i_alpha in trange(len(alpha)) for i in range(len(power_g)) ])
# laser_green_1=np.array(laser_green_1).reshape(len(alpha),len(power_g),len(guess))

#### Ionization and Recombination rate

In [ ]:
#only green
ng1=np.squeeze(laser_green)[:,0];ne1=np.squeeze(laser_green)[:,1]
ng2=np.squeeze(laser_green)[:,2];ne2=np.squeeze(laser_green)[:,3]

#975nm
ng1_975=np.squeeze(laser_975)[:,:,0];ne1_975=np.squeeze(laser_975)[:,:,1]
ng2_975=np.squeeze(laser_975)[:,:,2];ne2_975=np.squeeze(laser_975)[:,:,3]

#1064nm
ng1_1064=np.squeeze(laser_1064)[:,:,0];ne1_1064=np.squeeze(laser_1064)[:,:,1]
ng2_1064=np.squeeze(laser_1064)[:,:,2];ne2_1064=np.squeeze(laser_1064)[:,:,3]

In [ ]:
s_g1,s_e1,s_g2,s_e2,s_r1,s_r1_p,s_r2,s_r2_p,s_alpha1,s_alpha2,s_k1,s_k2,s_beta1,s_beta2,s_l1,s_l2,s_gamma1,s_gamma2,s_delta,s_I_g,s_IR_1064,s_IR_975= sympy.symbols('s_g1 s_e1 s_g2 s_e2 s_r1 s_r1_p s_r2 s_r2_p s_alpha1 s_alpha2 s_k1 s_k2 s_beta1 s_beta2 s_l1 s_l2 s_gamma1 s_gamma2 s_delta s_I_g s_IR_1064 s_IR_975')

In [ ]:
# Ionization and Recombination rate only for green
ion_g = ((s_r1*s_I_g*s_g1)-(s_r1_p*s_e1)+(s_alpha1*s_I_g*s_e1))/(s_e1+s_g1)
rec_g = ((s_r2*s_I_g*s_g2)-(s_r2_p*s_e2)+(s_alpha2*s_I_g*s_e2))/(s_e2+s_g2)

# Ionization and Recombination rate only for green + 975nm
ion_975 = (((s_r1*s_I_g+s_l1*s_IR_975*s_IR_975+s_delta*s_IR_975*s_IR_975)*s_g1)-(s_r1_p*s_e1)+((s_alpha1*s_I_g+s_gamma1*s_IR_975)*s_e1))/(s_e1+s_g1)
rec_975 = (((s_r2*s_I_g+s_l2*s_IR_975*s_IR_975)*s_g2)-(s_r2_p*s_e2)+((s_alpha2*s_I_g+s_gamma2*s_IR_975)*s_e2))/(s_e2+s_g2)

# Ionization and Recombination rate only for green + 1064nm
ion_1064 = (((s_r1*s_I_g+s_k1*s_IR_1064*s_IR_1064)*s_g1)-(s_r1_p*s_e1)+((s_alpha1*s_I_g+s_beta1*s_IR_1064)*s_e1))/(s_e1+s_g1)
rec_1064 = (((s_r2*s_I_g+s_k2*s_IR_1064*s_IR_1064)*s_g2)-(s_r2_p*s_e2)+((s_alpha2*s_I_g+s_beta2*s_IR_1064)*s_e2))/(s_e2+s_g2)
rec_975

### <span style= 'color: Green'> $\textbf{Comparison}$ </span> of Simulation and the Experiment

###### Getting errorBars for Charge state ratio

In [ ]:
# fig = go.Figure()
# fig.add_scatter(x=power_green_new,y=scaled_intensity_green,mode='markers',name='Exp. 532nm',marker=dict(color=px.colors.qualitative.Vivid[0],size=8,line=dict(color=px.colors.qualitative.Vivid[0],width=1)),
#     error_y=dict(type='data', array= std_error*scaled_intensity_green,visible=True,color=px.colors.qualitative.Vivid[0]),opacity=0.7)
# fig.add_scatter(x=power_g,y=pl_green_1,mode="lines",line=dict(width=2,color=px.colors.qualitative.Vivid[0]),name='Simulated 532nm')
# fig.add_annotation(x=48, y=12e6,
#             text="532nm",
#             showarrow=True,
#             arrowhead=3)
# #Set4
# pl_975_1=np.ones((pl_975.shape))
# for i in range(len(power_975_four)):
#     # if power_ir[i]*1e-3==50:
#     #     continue
#     # scale_set_975 = fun.interpolation(power_green_new,scale_two, power_975_four[i])
#     # fig.add_scatter(x=fun.origin(power_975_four[i]),y=fun.origin(intensity_975_four_scaled[i]),name=f'Exp. Scaled 975nm power : {power_ir[i]} mW ',mode="markers",line=dict(width=3,color=px.colors.qualitative.Vivid[i]))
#     fig.add_scatter(x=power_975_four[i],y=green4_ref_correct*intensity_975_four_rescaled[i],name=f'Exp. 975nm power : {power_ir[i]*1e-3} mW ',mode='markers',
#                     marker=dict(color=px.colors.qualitative.Vivid[i+2],size=8,line=dict(color=px.colors.qualitative.Vivid[i+2],width=1)),
#     error_y=dict(type='data', array=std_error*green4_ref_correct*intensity_975_four_rescaled[i],visible=True,color=px.colors.qualitative.Vivid[i+2]),)

#     pl_975_1[:,i] = pl_975[:,i]*pl_factor
#     fig.add_scatter(x=power_g,y=pl_975_1[:,i],name=f'Simulated 975nm power : {ir_powers_new[i]} mW ',mode="lines",line=dict(width=2,color=px.colors.qualitative.Vivid[i+2]))


# #Set2
# pl_1064_1=np.ones((pl_1064.shape))
# for i in range(len(power_1064_new)):
#     # if power_ir[i]*1e-3==50:
#     #     continue
#     # scale_set_1064 = fun.interpolation(power_green_new,scale_two, power_1064_new[i])
#     fig.add_scatter(x=power_1064_new[i],y=intensity_1064_new_scaled[i],name=f'Exp. 1064nm power : {power_ir[i]*1e-3} mW ',mode="markers",
#             marker=dict(color=px.colors.qualitative.Vivid[i+6],size=8,line=dict(color=px.colors.qualitative.Vivid[i+6],width=1)),
#     error_y=dict(type='data', array=std_error*intensity_1064_new_scaled[i] ,visible=True,color=px.colors.qualitative.Vivid[i+6]),)

#     pl_1064_1[:,i] = pl_1064[:,i]*pl_factor
#     fig.add_scatter(x=power_g,y=pl_1064_1[:,i],name=f'Simulated 1064nm power : {ir_powers_new[i]} mW ',mode="lines",line=dict(width=2,color=px.colors.qualitative.Vivid[i+6]))


# fig.update_layout(template=fig_template,height=500, width=1300,xaxis = dict(title = 'Green Power (\N{greek small letter mu}W)',range=[40,90]),yaxis = dict(title = 'Integrated PL' ))
# fig

In [ ]:
std_error

In [ ]:
hist_green,edge_green=np.histogram(scaled_ratio_green, bins=50, range=np.array([3,4]), density=False, weights=None)
hist_1064,edge_1064=np.histogram(np.ravel(ratio_1064_new_scaled), bins=50, range=np.array([2,4]), density=False, weights=None)
hist_975,edge_975=np.histogram(np.ravel(ratio_975_four), bins=50, range=np.array([2,4]), density=False, weights=None)

In [ ]:
np.median(np.ravel(ratio_1064_new_scaled)),np.median(np.ravel(ratio_975_four)),np.median(scaled_ratio_green),np.std(np.ravel(ratio_1064_new_scaled)),np.std(np.ravel(ratio_975_four)),np.std(scaled_ratio_green)

In [ ]:
fig = go.Figure()
fig.add_bar(x=edge_green,y=hist_green,name='532nm',marker=dict(color='Green'),showlegend=True,)
fig.add_bar(x=edge_1064,y=hist_1064,name='1064nm',marker=dict(color='orange'),showlegend=True,)
fig.add_bar(x=edge_975,y=hist_975,name='975nm',marker=dict(color='Red'),showlegend=True,)
fig.update_layout(template=fig_template,height=500, width=1000,xaxis = dict(title = 'Charge Ratio'),yaxis = dict(title = 'Counts' ),bargap=0.2)
fig

In [ ]:
# Gaussing distribution
def gauss(x,a0=1,x0=0,w0=1):
    beta = w0*np.sqrt(np.pi/np.log(2))/2
    f = a0*np.exp(-np.pi*(x-x0)**2/beta**2)
    return f

In [ ]:
hist_975,x_975 =np.histogram(np.ravel(ratio_975_four),
                             bins=17,
                             range=[2.5,3.8],
                            )
x_975 = fun.moving_average(x_975,2)

p0 = [3,3,0.3]
bounds = ([2,2.8,0.1],[6,3.2,0.6])

params_975 = curve_fit(gauss, x_975, hist_975, p0=p0, bounds=bounds)
params0_975 = params_975[0]

fit_975 = gauss(x_975,params0_975[0],params0_975[1],params0_975[2])

In [ ]:
hist_1064,x_1064 =np.histogram(np.ravel(ratio_1064_new_scaled),
                             bins=18,
                             range=[1.9,3.6],
                            )
x_1064 = fun.moving_average(x_1064,2)

p0 = [3,3,0.3]
bounds = ([2,2.5,0.1],[6,3.2,0.8])

params_1064 = curve_fit(gauss, x_1064, hist_1064, p0=p0, bounds=bounds)
params0_1064 = params_1064[0]

fit_1064 = gauss(x_1064,params0_1064[0],params0_1064[1],params0_1064[2])

In [ ]:
hist_green,x_green =np.histogram(scaled_ratio_green,
                             bins=14,
                             range=[3.3,4.6],
                            )
x_green = fun.moving_average(x_green,2)

p0 = [3,3.5,0.5]
bounds = ([1.5,3.5,0.4],[6,4.5,1])

params_green = curve_fit(gauss, x_green, hist_green, p0=p0, bounds=bounds)
params0_green = params_green[0]

fit_green = gauss(x_green,params0_green[0],params0_green[1],params0_green[2])

In [ ]:
params0_975, params0_1064, params0_green

In [ ]:
fig = go.Figure()
fig.add_bar(x=x_975,y=hist_975,name='975nm',marker=dict(color='Red'),showlegend=True,)
fig.add_scatter(x=x_975,y=fit_975,name='975nm',mode='lines',line=dict(color='black'),showlegend=True,)
fig.update_layout(template=fig_template,height=500, width=1000,xaxis = dict(title = 'Charge Ratio'),yaxis = dict(title = 'Counts' ),bargap=0.2)
fig

In [ ]:
fig = go.Figure()
fig.add_bar(x=x_1064,y=hist_1064,name='1064nm',marker=dict(color='Orange'),showlegend=True,)
fig.add_scatter(x=x_1064,y=fit_1064,name='1064nm',mode='lines',line=dict(color='black'),showlegend=True,)
fig.update_layout(template=fig_template,height=500, width=1000,xaxis = dict(title = 'Charge Ratio'),yaxis = dict(title = 'Counts' ),bargap=0.2)
fig

In [ ]:
fig = go.Figure()
fig.add_bar(x=x_green,y=hist_green,name='green',marker=dict(color='green'),showlegend=True,)
fig.add_scatter(x=x_green,y=fit_green,name='green',mode='lines',line=dict(color='black'),showlegend=True,)
fig.update_layout(template=fig_template,height=500, width=1000,xaxis = dict(title = 'Charge Ratio'),yaxis = dict(title = 'Counts' ),bargap=0.2)
fig

In [ ]:
hist_975s,x_975s =np.histogram(np.ravel(ratio_975_simul),
                             bins=25,
                             range=[3.1,3.25],
                            )
x_975s = fun.moving_average(x_975s,2)

p0 = [20,3.15,0.02]
bounds = ([5,2.9,0.01],[25,3.5,0.3])

params_975s = curve_fit(gauss, x_975s, hist_975s, p0=p0, bounds=bounds)
params0_975s = params_975s[0]

fit_975s = gauss(x_975s,params0_975s[0],params0_975s[1],params0_975s[2])

In [ ]:
params0_975s

In [ ]:
fig = go.Figure()
fig.add_bar(x=x_975s,y=hist_975s,name='975nm Simulation',marker=dict(color='Red'),showlegend=True,)
fig.add_scatter(x=x_975s,y=fit_975s,name='975nm Simulation',mode='lines',line=dict(color='black'),showlegend=True,)
fig.update_layout(template=fig_template,height=500, width=1000,xaxis = dict(title = 'Charge Ratio'),yaxis = dict(title = 'Counts' ),bargap=0.2)
fig

In [ ]:
# Green
np.mean(scaled_ratio_green), np.std(scaled_ratio_green)

In [ ]:
# 975 nm Simulations
np.mean(ratio_975_simul), np.std(ratio_975_simul)

In [ ]:
# 1064 nm Simulations
np.mean(ratio_1064_simul), np.std(ratio_1064_simul)

In [ ]:
# Green Simulations
np.mean(ratio_green_simul), np.std(ratio_green_simul)

In [ ]:
charge_ratio_exp = [np.mean(scaled_ratio_green),params0_975[1], params0_1064[1]]
charge_ratio_sim = [np.mean(ratio_green_simul),np.mean(ratio_975_simul), np.mean(ratio_1064_simul)]

charge_ratio_exp_std = [np.std(scaled_ratio_green),params0_975[2]/2, params0_1064[2]/2]
charge_ratio_sim_std = [np.std(ratio_green_simul),np.std(ratio_975_simul), np.std(ratio_1064_simul)]

In [ ]:
charge_ratio_exp_std

In [ ]:
np.squeeze(ionization_1064),np.squeeze(recombination_1064)

In [ ]:
np.squeeze(ionization_green),np.squeeze(recombination_green)

In [ ]:
# ionization rate comparison 1064 nm
ion_ratio_1064 = (np.squeeze(ionization_1064)/(np.tile(np.squeeze(ionization_green),(4,1)).T))[:,-1]

In [ ]:
# recombination rate comparison 1064 nm
rec_ratio_1064 = (np.squeeze(recombination_1064)/(np.tile(np.squeeze(recombination_green),(4,1)).T))[:,-1]

In [ ]:
ion_ratio_1064,rec_ratio_1064

In [ ]:
# ionization rate comparison 975 nm
ion_ratio_975 = (np.squeeze(ionization_975)/(np.tile(np.squeeze(ionization_green),(4,1)).T))[:,-1]

In [ ]:
# recombination rate comparison 975 nm
rec_ratio_975 = (np.squeeze(recombination_975)/(np.tile(np.squeeze(recombination_green),(4,1)).T))[:,-1]

In [ ]:
pl_ratio_ir_sim = np.mean(pl_975_1/pl_1064_1,axis=0)

In [ ]:
pl_ratio_ir_exp = np.mean((green4_ref_correct*intensity_975_four_rescaled/intensity_1064_new_scaled).T,axis=0)

In [ ]:
pl_ratio_ir_exp_std = np.std((green4_ref_correct*intensity_975_four_rescaled/intensity_1064_new_scaled).T,axis=0)

In [ ]:
# fig = go.Figure()
# fig.add_scatter(x=power_ir*1e-3,y=pl_ratio_ir_exp,mode='markers',marker=dict(color='blue',size=12,line=dict(color='black',width=1)),
#     error_y=dict(type='data', array=pl_ratio_ir_exp_std ,visible=True,color='black'),name='Experimental',opacity=0.7)
# fig.add_scatter(x=power_ir*1e-3,y=pl_ratio_ir_sim,mode='lines',line=dict(width=3,color='red'),name='Simulation',marker=dict(
#             color='LightSkyBlue',
#             symbol="diamond",
#             size=20,
#             line=dict(
#                 color='MediumPurple',
#                 width=2
#             )
#         ),opacity=0.9)
# fig.update_layout(template=fig_template,height=500, width=1000,xaxis = dict(title = 'IR Power mW',range=[10,110]),yaxis = dict(title = '<sup>PL<sub>975</sub></sup>&#8260;<sub>PL<sub>1064</sub></sub>' ))
# fig

In [ ]:
data = {'power_g': power_g,
        'ion_975': ion_ratio_975,
        'ion_1064': ion_ratio_1064,
        'rec_975': rec_ratio_975,
        'rec_1064': rec_ratio_1064}

df = pd.DataFrame(data)

In [ ]:
pathfig = path+"\\images\\"

In [ ]:
fig = go.Figure()
fig.add_scatter(x=power_green_new,y=scaled_intensity_green/1e6,mode='markers',name=False,marker=dict(color=vivid[3],size=8+0.3,line=dict(color=vivid[3],width=1+0.3)),
    error_y=dict(type='percent', value= std_error*100,visible=True,color=vivid[3],thickness=2,width=3),opacity=0.7,showlegend=False)
fig.add_scatter(x=power_g,y=pl_green_1/1e6,mode="lines",line=dict(width=2+0.3,color=vivid[3]),showlegend=False)

# fig.add_annotation(text="532nm ",yref='paper',xref='paper', x=0.18,y=0.51,
#             showarrow=True,
#             arrowhead=3)

#Set2
pl_1064_1=np.ones((pl_1064.shape))
for i in range(len(power_1064_new)):
    # if power_ir[i]*1e-3==50:
    #     continue
    # scale_set_1064 = fun.interpolation(power_green_new,scale_two, power_1064_new[i])
    fig.add_scatter(x=power_1064_new[i],y=intensity_1064_new_scaled[i]/1e6,name=f'{power_ir[i]*1e-3} mW ',mode="markers",
            marker=dict(color=vivid[i+6],size=8+0.3,line=dict(color=vivid[i+6],width=1+0.3)),
    error_y=dict(type='percent', value= std_error*100,visible=True,color=vivid[i+6],thickness=2,width=3),)

    pl_1064_1[:,i] = pl_1064[:,i]*pl_factor
    fig.add_scatter(x=power_g,y=pl_1064_1[:,i]/1e6,mode="lines",line=dict(width=2+0.5,color=vivid[i+6]),showlegend=False)


fig.update_layout( # customize font and legend orientation & position
    legend=dict(
        title='', orientation="h", y=1.05, yanchor="bottom", x=0.5, xanchor="center",title_font_family="Times New Roman",
    )
)
# fig.add_annotation( # add a text callout with arrow
#     text="1064nm : ",yref='paper',xref='paper', x=0.25,y=1.17, showarrow=False,
#     font_family="Times New Roman",textangle=0
# )
fig.update_yaxes(title_font_family="Times New Roman");fig.update_xaxes(title_font_family="Times New Roman")


fig.update_layout(template=fig_template,height=450, width=600,xaxis = dict(title = 'Green Power (\N{greek small letter mu}W)',range=[40,90]),yaxis = dict(title = "Integrated PL (arb. u.) ",range=[0,22e6/1e6] ))
fig.write_image(pathfig+"fig3_1064pl.pdf",engine='kaleido')
fig.write_html(pathfig+"fig3_1064pl.html")
fig.show()

##### PLots for the Paper

In [ ]:
pl_green_average = fun.data_averaging(data_green_new[:,0],data_green_four[:,0],data_green_scaled[:,-1],data_green_scaled_four[:,-1]*(green4_ref_correct[-1]*green_scale_four_two[-1]))

In [ ]:
#green and 1064nm from the Second dataSets
fig = go.Figure()

# fig.add_scatter(x=data_green_four[:,0],y=data_green_scaled_four[:,-1]*(green4_ref_correct[-1]*green_scale_four_two[-1]),marker=dict(color=vivid[3],size=8+0.3,line=dict(color=vivid[3],width=1+0.3)),showlegend=False,opacity= 0.3)
# fig.add_scatter(x=data_green_new[:,0],y=data_green_scaled[:,-1],marker=dict(color=vivid[3],size=8+0.3,line=dict(color=vivid[3],width=1+0.3)),showlegend=False)
fig.add_scatter(x=data_green_new[:,0],y=pl_green_average/1e3,marker=dict(color=vivid[3],size=8+0.3,line=dict(color=vivid[3],width=1+0.3)),showlegend=False)


irs = ['1064']
for i_ir in irs:
    for j in range(len(ir_powers_new)):
        fig.add_scatter(x=globals()[f'data_{i_ir}_new_scaled_'+f'{ir_powers_new[j]}'][:,0],y=(globals()[f'data_{i_ir}_new_scaled_'+f'{ir_powers_new[j]}'][:,-1])/1e3 ,marker=dict(color=vivid[j+6],size=8+0.3,line=dict(color=vivid[j+6],width=1+0.3)),showlegend=False )

fig.update_layout(template=fig_template,height=400, width=500,xaxis = dict(title = '',range=[555,785]),yaxis = dict(title = '' ,range=[0,60]))
# fig.update_layout(template=fig_template,height=450, width=600,xaxis = dict(title = 'Wavelength (&#955;) '),yaxis = dict(title = "Intensity ( &times;10<sup>3</sup>) " ))
fig.update_layout(template=fig_template,height=450, width=600,xaxis = dict(title = 'Wavelength (nm) '),yaxis = dict(title = "Intensity (arb. u.) " ))


fig.update_yaxes(title_font_family="Times New Roman");fig.update_xaxes(title_font_family="Times New Roman")
fig.write_image(pathfig+"fig3_1064_inset.pdf",engine='kaleido')
fig.write_image(pathfig+"fig3_1064_inset.png")
fig.write_html(pathfig+"fig3_1064_inset_50muW.html")
fig.show()

In [ ]:
fig = go.Figure()
fig.add_scatter(x=power_green_new,y=scaled_intensity_green/1e6,mode='markers',name=False,marker=dict(color=vivid[3],size=8+0.3,line=dict(color=vivid[3],width=+0.31)),
    error_y=dict(type='percent', value= std_error*100,visible=True,color=vivid[3]),opacity=0.7,showlegend=False)
fig.add_scatter(x=power_g,y=pl_green_1/1e6,mode="lines",line=dict(width=2+0.3,color=vivid[3]),showlegend=False)

# fig.add_annotation(text="532nm ",yref='paper',xref='paper', x=0.18,y=0.51,
#             showarrow=True,
#             arrowhead=3)

#Set4
pl_975_1=np.ones((pl_975.shape))
for i in range(len(power_975_four)):
    fig.add_scatter(x=power_975_four[i],y=green4_ref_correct*intensity_975_four_rescaled[i]/1e6,name=f'{power_ir[i]*1e-3} mW ',mode='markers',
                    marker=dict(color=vivid[i+6],size=8+0.3,line=dict(color=vivid[i+6],width=1+0.3)),
    error_y=dict(type='percent', value= std_error*100,visible=True,color=vivid[i+6]),)

    pl_975_1[:,i] = pl_975[:,i]*pl_factor
    fig.add_scatter(x=power_g,y=pl_975_1[:,i]/1e6,mode="lines",line=dict(width=2+0.3,color=vivid[i+6]),showlegend=False)


fig.update_layout( # customize font and legend orientation & position
    legend=dict(
        title='', orientation="h", y=1.05, yanchor="bottom", x=0.68, xanchor="center",title_font_family="Times New Roman",
    )
)
# fig.add_annotation( # add a text callout with arrow
#     text="975nm : ",yref='paper',xref='paper', x=0.25,y=1.17, showarrow=False,
#     font_family="Times New Roman",textangle=0
# )
fig.update_yaxes(title_font_family="Times New Roman");fig.update_xaxes(title_font_family="Times New Roman")


fig.update_layout(template=fig_template,height=450, width=600,xaxis = dict(title = 'Green Power (\N{greek small letter mu}W)',range=[40,90]),yaxis = dict(title = 'Integrated PL', range=[0,22e6/1e6]))
fig.update_layout(template=fig_template,height=450, width=600,xaxis = dict(title = 'Green Power (\N{greek small letter mu}W)',range=[40,90]),yaxis = dict(title = "Integrated PL (arb. u.) ",range=[0,22e6/1e6] ))

fig.write_image(pathfig+"fig3_975pl.pdf",engine='kaleido')
fig.write_html(pathfig+"fig3_975pl.html")
fig

In [ ]:
fig = go.Figure()

# fig.add_scatter(x=data_green_four[:,0],y=data_green_scaled_four[:,-1]*(green4_ref_correct[-1]*green_scale_four_two[-1]),marker=dict(color=vivid[3],size=8+0.3,line=dict(color=vivid[3],width=1+0.3)),showlegend=False,opacity= 0.3)
# fig.add_scatter(x=data_green_new[:,0],y=data_green_scaled[:,-1],marker=dict(color=vivid[3],size=8+0.3,line=dict(color=vivid[3],width=1+0.3)),showlegend=False)
fig.add_scatter(x=data_green_new[:,0],y=pl_green_average/1e3,marker=dict(color=vivid[3],size=8+0.3,line=dict(color=vivid[3],width=1+0.3)),showlegend=False)



irs = ['975']
for i_ir in irs:
    for j in range(len(ir_powers_four)):
            fig.add_scatter(x=globals()[f'data_{i_ir}_four_scaled_'+f'{ir_powers_four[j]}'][:,0],y=(globals()[f'data_{i_ir}_four_scaled_'+f'{ir_powers_four[j]}'][:,-1]*(green4_ref_correct[-1]*green_scale_four_two[-1]))/1e3 ,showlegend=False,
                           marker=dict(color=vivid[j+6],size=8+0.3,line=dict(color=vivid[j+6],width=1+0.3)),)

fig.update_layout(template=fig_template,height=400, width=500,xaxis = dict(title = '',range=[555,785]),yaxis = dict(title = '' ,range=[0,60]))
fig.update_layout(template=fig_template,height=450, width=600,xaxis = dict(title = 'Wavelength (nm) '),yaxis = dict(title = "Intensity (arb. u.)" ))

fig.update_yaxes(title_font_family="Times New Roman");fig.update_xaxes(title_font_family="Times New Roman")
fig.write_image(pathfig+"fig3_975_inset.pdf",engine='kaleido')
fig.write_image(pathfig+"fig3_975_inset.png")
fig.write_html(pathfig+"fig3_975_inset_83muW.html")
fig.show()

In [ ]:
x_laser = ['20mW', '50mW','70mW','100mW']
fig = go.Figure()
fig.add_bar(x=x_laser,y=pl_ratio_ir_exp,marker=dict(color='royalblue',line=dict(width=1.0+0.5,color='black')),error_y=dict(type='data', array=pl_ratio_ir_exp_std ,visible=True,color='black',thickness=1.5+0.3,width=3+0.3),name='Experimental',
        # text = x_laser,
        # textposition='auto',textangle=-20,insidetextanchor='middle',
        # textfont=dict(
        # size=20,
        # color="#ffffff"),
        marker_color=[vivid[6],vivid[7],vivid[8],vivid[9]],opacity=0.67)

# fig.add_scatter(x=x_laser,y=pl_ratio_ir_sim,mode='markers',line=dict(width=1,color=vivid[5]),name='Simulation',marker=dict(
#             color=pastel[6],
#             symbol="diamond-open-dot",
#             size=15,
#             line=dict(
#                 color='black',
#                 width=1.5
#             ),opacity=0.9
#         ),opacity=0.9)
fig.add_scatter(x=x_laser,y=pl_ratio_ir_sim,mode="markers",name='Simulation',marker=dict(color='white',symbol="diamond-dot", opacity=1, size=14,line=dict(color='black',width=1+0.5)),opacity=0.7)


fig.update_layout(template=fig_template,height=450, width=600,xaxis = dict(title = 'IR Power'),yaxis = dict(title = '<sup>PL<sub>975</sub></sup>&#8260;<sub>PL<sub>1064</sub></sub>' ,range=[0.5,4.5]))

# fig.update_layout(legend_title_text='Financials',
#                  title_text='2009-2019 Financial Report')

fig.update_layout( # customize font and legend orientation & position
    legend=dict(
        title=None, orientation="h", y=0.8, yanchor="bottom", x=0.4, xanchor="center",title_font_family="Times New Roman",
    )
)

fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')
# fig.update_xaxes(showticklabels=False)
fig.update_xaxes(tickangle=15)
fig.update_yaxes(title_font_family="Times New Roman");fig.update_xaxes(title_font_family="Times New Roman")
fig.write_image(pathfig+"fig3_IR_pl_ratio.pdf",engine='kaleido')
fig

In [ ]:
x_laser = ['532nm', '532nm <br> + 975nm','532nm <br>+ 1064nm']
fig = go.Figure()
fig.add_bar(x=x_laser,y=charge_ratio_exp,marker=dict(color='royalblue',line=dict(width=1.5,color='black')),error_y=dict(type='data', array=charge_ratio_exp_std ,visible=True,color='black',thickness=1.5+0.3,width=3+0.3),name='Experimental',
        # text = x_laser,
        # textposition='outside',textangle=20,
            # insidetextanchor='middle',
        # textfont=dict(
        # size=20,
        # color="black"),
        marker_color=[vivid[3],g10[2],g10[8]],opacity=0.67)
fig.add_scatter(x=x_laser,y=charge_ratio_sim,mode="markers",name='Simulation',marker=dict(color='white',symbol="diamond-dot", opacity=1, size=14,line=dict(color='black',width=1.5)),opacity=0.7)

fig.update_layout(template=fig_template,height=450, width=600,yaxis = dict(title = '<b><sup>PL<sub>NV<sup>-</sup></sub></sup>&#8260;<sub>PL<sub>NV<sup>0</sup></sub></sub></b>', range=[2.2,4.3]))

# fig.update_layout(legend_title_text='Financials',
#                  title_text='2009-2019 Financial Report')

fig.update_layout( # customize font and legend orientation & position
    legend=dict(
        title=None, orientation="h", y=0.8, yanchor="bottom", x=0.6, xanchor="center",title_font_family="Times New Roman",
    )
)

fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')
# fig.update_xaxes(showticklabels=False)
fig.update_xaxes(tickangle=0)
fig.update_yaxes(title_font_family="Times New Roman");fig.update_xaxes(title_font_family="Times New Roman")
fig.write_image(pathfig+"fig4_NV_pl_ratio.pdf",engine='kaleido')
fig

In [ ]:
fig = go.Figure()
fig.add_scatter(x=power_g,y=np.squeeze(ionization_green),mode='lines',line=dict(width=2+0.5,color=vivid[3]),marker=dict(
            color='LightSkyBlue',
            symbol="triangle-right-dot",
            size=12,
            line=dict(
                color='MediumPurple',
                width=1.0+0.3
            ),opacity=0.9
        ),opacity=1.0,name='532 nm')
fig.add_scatter(x=power_g,y=np.squeeze(ionization_975)[:,-1],mode='lines',line=dict(width=2+0.5,color=g10[2]),marker=dict(
            color=g10[2],
            symbol="triangle-right-dot",
            size=12,
            line=dict(
                color=g10[2],
                width=1.0+0.3
            ),opacity=0.9
        ),opacity=1.0,name='975 nm')
fig.add_scatter(x=power_g,y=np.squeeze(ionization_1064)[:,-1],mode='lines',line=dict(width=2+0.5,color=g10[8]),marker=dict(
            color=plotly[0],
            symbol="triangle-right-dot",
            size=12,
            line=dict(
                color=g10[8],
                width=1.0+0.3
            ),opacity=0.9
        ),opacity=1.0,name='1064 nm')

fig.update_layout(template=fig_template,height=450, width=600,xaxis = dict(title = 'Green Power (&mu;W)',range=[40,90]),yaxis = dict(title = 'I<sub>eff</sub> (&times;10<sup>3</sup>s<sup>-1</sup>)'))

# fig.add_annotation( # add a text callout with arrow
#     text="Green <br> normalized",yref='paper',xref='paper', x=1.27,y=1.1, showarrow=False,
#     font_family="Droid Serif",textangle=0
# )

# fig.update_layout( # customize font and legend orientation & position
#     legend=dict(
#         title='', orientation="v", y=0.25, yanchor="bottom", x=1.2, xanchor="center",bordercolor="Black",borderwidth=0.2
#     )
# )

# fig.update_layout( # customize font and legend orientation & position
#     legend=dict(
#         title=None, orientation="h", y=0.8, yanchor="bottom", x=0.73, xanchor="center",title_font_family="Times New Roman",
#     )
# )
# fig.update_layout(showlegend=False)
fig.write_image(pathfig+"fig4_ionization_green.pdf",engine='kaleido')

fig

In [ ]:
def curve_func(x,a,m):
    return m*pow(x,a)

In [ ]:
paras_green_ion = curve_fit(curve_func, power_g, np.squeeze(ionization_green))
paras_975_ion_100 = curve_fit(curve_func, power_g, np.squeeze(ionization_975)[:,-1])
paras_1064_ion_100 = curve_fit(curve_func, power_g, np.squeeze(ionization_1064)[:,-1])
paras_975_ion_70 = curve_fit(curve_func, power_g, np.squeeze(ionization_975)[:,-2])
paras_1064_ion_70 = curve_fit(curve_func, power_g, np.squeeze(ionization_1064)[:,-2])
paras_975_ion_50 = curve_fit(curve_func, power_g, np.squeeze(ionization_975)[:,-3])
paras_1064_ion_50 = curve_fit(curve_func, power_g, np.squeeze(ionization_1064)[:,-3])
paras_975_ion_20 = curve_fit(curve_func, power_g, np.squeeze(ionization_975)[:,-4])
paras_1064_ion_20 = curve_fit(curve_func, power_g, np.squeeze(ionization_1064)[:,-4])

paras_green_rec = curve_fit(curve_func, power_g, np.squeeze(recombination_green))
paras_975_rec_100 = curve_fit(curve_func, power_g, np.squeeze(recombination_975)[:,-1])
paras_1064_rec_100 = curve_fit(curve_func, power_g, np.squeeze(recombination_1064)[:,-1])
paras_975_rec_70 = curve_fit(curve_func, power_g, np.squeeze(recombination_975)[:,-2])
paras_1064_rec_70 = curve_fit(curve_func, power_g, np.squeeze(recombination_1064)[:,-2])
paras_975_rec_50 = curve_fit(curve_func, power_g, np.squeeze(recombination_975)[:,-3])
paras_1064_rec_50 = curve_fit(curve_func, power_g, np.squeeze(recombination_1064)[:,-3])
paras_975_rec_20 = curve_fit(curve_func, power_g, np.squeeze(recombination_975)[:,-4])
paras_1064_rec_20 = curve_fit(curve_func, power_g, np.squeeze(recombination_1064)[:,-4])

In [ ]:
np.round(paras_green_ion[0],3),np.round(paras_green_rec[0],3)    ### (a,m) in function : m*pow(x,a)

In [ ]:
np.round(paras_975_ion_100[0],3),np.round(paras_975_ion_70[0],3),np.round(paras_975_ion_50[0],3),np.round(paras_975_ion_20[0],3)

In [ ]:
np.round(paras_975_rec_100[0],3),np.round(paras_975_rec_70[0],3),np.round(paras_975_rec_50[0],3),np.round(paras_975_rec_20[0],3)

In [ ]:
np.round(paras_1064_ion_100[0],3),np.round(paras_1064_ion_70[0],3),np.round(paras_1064_ion_50[0],3),np.round(paras_1064_ion_20[0],3)

In [ ]:
np.round(paras_1064_rec_100[0],3),np.round(paras_1064_rec_70[0],3),np.round(paras_1064_rec_50[0],3),np.round(paras_1064_rec_20[0],3)

In [ ]:
print('Fitting the function "m*x^a" in the Ionization rates and Recombination rates (with green Power in the xaxis) \nfor different lasers with increasing order of power, the values for (a,m): \n')
print(f'for 532nm, Ionization rate: {np.round(paras_green_ion[0],3)}\t Recombination rate: {np.round(paras_green_rec[0],3)}')    ### (a,m) in function : m*pow(x,a)
print(f'for 975nm,\n Ionization rate: {np.round(paras_975_ion_20[0],3)},{np.round(paras_975_ion_50[0],3)},{np.round(paras_975_ion_70[0],3)},{np.round(paras_975_ion_100[0],3)}')
print(f' Recombination rate: {np.round(paras_975_rec_20[0],3)},{np.round(paras_975_rec_50[0],3)},{np.round(paras_975_rec_70[0],3)},{np.round(paras_975_rec_100[0],3)}')
print(f'\nfor 1064nm,\n Ionization rate: {np.round(paras_1064_ion_20[0],3)},{np.round(paras_1064_ion_50[0],3)},{np.round(paras_1064_ion_70[0],3)},{np.round(paras_1064_ion_100[0],3)}')
print(f' Recombination rate: {np.round(paras_1064_rec_20[0],3)},{np.round(paras_1064_rec_50[0],3)},{np.round(paras_1064_rec_70[0],3)},{np.round(paras_1064_rec_100[0],3)}')

In [ ]:
fig = go.Figure()

fig.add_scatter(x=power_g,y=np.squeeze(ionization_green),mode='lines',line=dict(width=2+0.5,color=vivid[3]),marker=dict(
            color='LightSkyBlue',
            symbol="triangle-right-dot",
            size=12,
            line=dict(
                color='MediumPurple',
                width=1.0+0.3
            ),opacity=0.9
        ),opacity=1.0,name='532 nm')

fig.add_scatter(x=power_g,y=np.squeeze(ionization_975)[:,-1],mode='lines',line=dict(width=2+0.5,color=g10[2]),marker=dict(
            color=g10[2],
            symbol="triangle-right-dot",
            size=12,
            line=dict(
                color=g10[2],
                width=1.0+0.3
            ),opacity=0.9
        ),opacity=1.0,name='975 nm')

fig.add_scatter(x=power_g,y=np.squeeze(ionization_1064)[:,-1],mode='lines',line=dict(width=2+0.5,color=g10[8]),marker=dict(
            color=plotly[0],
            symbol="triangle-right-dot",
            size=12,
            line=dict(
                color=g10[8],
                width=1.0+0.3
            ),opacity=0.9
        ),opacity=1.0,name='1064 nm')

fig.update_layout(template=fig_template,height=450, width=600,xaxis = dict(title = 'Green Power (&mu;W)',range=[40,90]),yaxis = dict(title = 'I<sub>eff</sub>  ( &times; 10<sup>3</sup>s<sup>-1</sup> )'))

# fig.add_annotation( # add a text callout with arrow
#     text="Green <br> normalized",yref='paper',xref='paper', x=1.27,y=1.1, showarrow=False,
#     font_family="Droid Serif",textangle=0
# )

# fig.update_layout( # customize font and legend orientation & position
#     legend=dict(
#         title='', orientation="v", y=0.25, yanchor="bottom", x=1.2, xanchor="center",bordercolor="Black",borderwidth=0.2
#     )
# )

fig.update_layout( # customize font and legend orientation & position
    legend=dict(
        title=None, orientation="h", y=1.1, yanchor="bottom", x=0.43, xanchor="center",title_font_family="Times New Roman",
    )
)
# fig.update_layout(showlegend=False)
fig.write_image(pathfig+"fig4_ionization_green.pdf",engine='kaleido')

fig

In [ ]:
fig = go.Figure()

fig.add_scatter(x=power_g,y=np.squeeze(ionization_green),mode='lines',line=dict(width=2+0.5,color=vivid[3]),marker=dict(
            color='LightSkyBlue',
            symbol="triangle-right-dot",
            size=12,
            line=dict(
                color='MediumPurple',
                width=1.0+0.3
            ),opacity=0.9
        ),opacity=1.0,name='532 nm')
fig.update_layout(template=fig_template,height=450, width=600,xaxis = dict(title = 'Green Power (&mu;W)',range=[40,90]),yaxis = dict(title = 'I<sub>eff</sub>  ( &times; 10<sup>3</sup>s<sup>-1</sup> )'))

# fig.add_annotation( # add a text callout with arrow
#     text="Green <br> normalized",yref='paper',xref='paper', x=1.27,y=1.1, showarrow=False,
#     font_family="Droid Serif",textangle=0
# )

# fig.update_layout( # customize font and legend orientation & position
#     legend=dict(
#         title='', orientation="v", y=0.25, yanchor="bottom", x=1.2, xanchor="center",bordercolor="Black",borderwidth=0.2
#     )
# )

fig.update_layout( # customize font and legend orientation & position
    legend=dict(
        title=None, orientation="h", y=0.8, yanchor="bottom", x=0.73, xanchor="center",title_font_family="Times New Roman",
    )
)
# fig.update_layout(showlegend=False)
fig.write_image(pathfig+"fig4_ionization_green_inset.pdf",engine='kaleido')

fig

In [ ]:
fig = go.Figure()
fig.add_scatter(x=power_g,y=np.squeeze(recombination_green),mode='lines',line=dict(width=2+0.5,color=vivid[3]),marker=dict(
            color='LightSkyBlue',
            symbol="triangle-right-dot",
            size=12,
            line=dict(
                color='MediumPurple',
                width=1.0+0.3
            ),opacity=0.9
        ),opacity=1.0,name='532 nm')
fig.add_scatter(x=power_g,y=np.squeeze(recombination_975)[:,-1],mode='lines',line=dict(width=2+0.5,color=g10[2]),marker=dict(
            color=g10[2],
            symbol="triangle-right-dot",
            size=12,
            line=dict(
                color=g10[2],
                width=1.0+0.3
            ),opacity=0.9
        ),opacity=1.0,name='975 nm')
fig.add_scatter(x=power_g,y=np.squeeze(recombination_1064)[:,-1],mode='lines',line=dict(width=2+0.5,color=g10[8]),marker=dict(
            color=plotly[0],
            symbol="triangle-right-dot",
            size=12,
            line=dict(
                color=g10[8],
                width=1.0+0.3
            ),opacity=0.9
        ),opacity=1.0,name='1064 nm')

fig.update_layout(template=fig_template,height=450, width=600,xaxis = dict(title = 'Green Power (&mu;W)',range=[40,90]),yaxis = dict(title = 'R<sub>eff</sub>  ( &times; 10<sup>3</sup>s<sup>-1</sup> )'))

# fig.add_annotation( # add a text callout with arrow
#     text="Green <br> normalized",yref='paper',xref='paper', x=1.27,y=1.1, showarrow=False,
#     font_family="Droid Serif",textangle=0
# )

# fig.update_layout( # customize font and legend orientation & position
#     legend=dict(
#         title='', orientation="v", y=0.25, yanchor="bottom", x=1.2, xanchor="center",bordercolor="Black",borderwidth=0.2
#     )
# )

fig.update_layout( # customize font and legend orientation & position
    legend=dict(
        title=None, orientation="h", y=1.1, yanchor="bottom", x=0.43, xanchor="center",title_font_family="Times New Roman",
    )
)
# fig.update_layout(showlegend=False)
fig.write_image(pathfig+"fig4_recombination_green.pdf",engine='kaleido')

fig

In [ ]:
fig = go.Figure()
fig.add_scatter(x=power_g,y=np.squeeze(recombination_green),mode='lines',line=dict(width=2+0.5,color=vivid[3]),marker=dict(
            color='LightSkyBlue',
            symbol="triangle-right-dot",
            size=12,
            line=dict(
                color='MediumPurple',
                width=1.0+0.3
            ),opacity=0.9
        ),opacity=1.0,name='532 nm')

fig.update_layout(template=fig_template,height=450, width=600,xaxis = dict(title = 'Green Power (&mu;W)',range=[40,90]),yaxis = dict(title = 'R<sub>eff</sub>  ( &times; 10<sup>3</sup>s<sup>-1</sup> )'))

# fig.add_annotation( # add a text callout with arrow
#     text="Green <br> normalized",yref='paper',xref='paper', x=1.27,y=1.1, showarrow=False,
#     font_family="Droid Serif",textangle=0
# )

# fig.update_layout( # customize font and legend orientation & position
#     legend=dict(
#         title='', orientation="v", y=0.25, yanchor="bottom", x=1.2, xanchor="center",bordercolor="Black",borderwidth=0.2
#     )
# )

fig.update_layout( # customize font and legend orientation & position
    legend=dict(
        title=None, orientation="h", y=0.8, yanchor="bottom", x=0.73, xanchor="center",title_font_family="Times New Roman",
    )
)
# fig.update_layout(showlegend=False)
fig.write_image(pathfig+"fig4_recombination_green_inset.pdf",engine='kaleido')

fig

In [ ]:
fig = go.Figure()
fig.add_scatter(x=df['power_g'],y=df['ion_1064'],mode='lines',line=dict(width=3+0.5,color=g10[8]),marker=dict(
            color='LightSkyBlue',
            symbol="triangle-right-dot",
            size=12,
            line=dict(
                color='MediumPurple',
                width=1.0+0.3
            ),opacity=0.9
        ),opacity=1.0,name='1064 nm')
fig.add_scatter(x=df['power_g'],y=df['ion_975'],mode='lines',line=dict(width=3+0.5,color=g10[2]),marker=dict(
            color=plotly[0],
            symbol="triangle-right-dot",
            size=12,
            line=dict(
                color=plotly[4],
                width=1.0+0.3
            ),opacity=0.9
        ),opacity=1.0,name='975 nm')

fig.update_layout(template=fig_template,height=450, width=600,xaxis = dict(title = 'Green Power (&mu;W)',range=[40,90]),yaxis = dict(title = 'Inoization Rate',range=[0,260]))

# fig.add_annotation( # add a text callout with arrow
#     text="Green <br> normalized",yref='paper',xref='paper', x=1.27,y=1.1, showarrow=False,
#     font_family="Droid Serif",textangle=0
# )

# fig.update_layout( # customize font and legend orientation & position
#     legend=dict(
#         title='', orientation="v", y=0.25, yanchor="bottom", x=1.2, xanchor="center",bordercolor="Black",borderwidth=0.2
#     )
# )

fig.update_layout( # customize font and legend orientation & position
    legend=dict(
        title=None, orientation="h", y=0.8, yanchor="bottom", x=0.73, xanchor="center",title_font_family="Times New Roman",
    )
)
# fig.update_layout(showlegend=False)
fig.write_image(pathfig+"fig4_ion.pdf",engine='kaleido')

fig

In [ ]:
fig = go.Figure()
fig.add_scatter(x=df['power_g'],y=df['rec_1064'],mode='lines',line=dict(width=3+0.5,color=g10[8]),marker=dict(
            color='LightSkyBlue',
            symbol="triangle-right-dot",
            size=12,
            line=dict(
                color='MediumPurple',
                width=1.0+0.3
            ),opacity=0.9
        ),opacity=1.0,name='1064 nm')
fig.add_scatter(x=df['power_g'],y=df['rec_975'],mode='lines',line=dict(width=3+0.5,color=g10[2]),marker=dict(
            color=plotly[0],
            symbol="triangle-right-dot",
            size=12,
            line=dict(
                color=plotly[4],
                width=1.0+0.3
            ),opacity=0.9
        ),opacity=1.0,name='975 nm')

fig.update_layout(template=fig_template,height=450, width=600,xaxis = dict(title = 'Green Power (&mu;W)',range=[40,90]),yaxis = dict(title = 'Recombination Rate',range=[0,260]))
fig.update_layout( # customize font and legend orientation & position
    legend=dict(
        title=None, orientation="h", y=0.8, yanchor="bottom", x=0.73, xanchor="center",title_font_family="Times New Roman",
    )
)

fig.write_image(pathfig+"fig4_rec.pdf",engine='kaleido')
fig

In [ ]:
std_error

In [ ]:
fig = go.Figure()
fig.add_bar(x=x_error,y=hist_error,name='intensity_error',marker=dict(line=dict(width=1.5,color='black')),marker_color=px.colors.qualitative.Set1[6],opacity=0.9)


fig.update_layout(template=fig_template,height=450, width=600,xaxis = dict(title = '', ))

fig.update_layout( # customize font and legend orientation & position
    legend=dict(
        title=None, orientation="h", y=0.8, yanchor="bottom", x=0.6, xanchor="center",title_font_family="Times New Roman",
    )
)

fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')
# fig.update_xaxes(showticklabels=False)
fig.update_xaxes(tickangle=0)
fig.update_yaxes(title_font_family="Times New Roman");fig.update_xaxes(title_font_family="Times New Roman")
fig.update_layout(template=fig_template,height=450, width=600,xaxis = dict(title = 'Integrated Photoluminescence (Normalized)'))

fig.add_annotation(x=0.85, y=5,
            text=f"&#963; &#8260;&#8730;n = {np.round(std_error,3)} ",
            showarrow=False,
            )
fig.write_image(pathfig+"errorbars.pdf",engine='kaleido')
fig.show()

In [ ]:
fig = go.Figure()
fig.add_scatter(x=num_error,y=ratio_error,name='ratio of errors (set1)',mode="markers",)

fig.update_layout( # customize font and legend orientation & position
    legend=dict(
        title=None, orientation="h", y=0.8, yanchor="bottom", x=0.6, xanchor="center",title_font_family="Times New Roman",
    )
)

fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')
# fig.update_xaxes(showticklabels=False)
fig.update_xaxes(tickangle=0)
fig.update_yaxes(title_font_family="Times New Roman");fig.update_xaxes(title_font_family="Times New Roman")
fig.update_layout(template=fig_template,height=450, width=600,xaxis = dict(title = 'Time (total length ~ 1 hour)'),yaxis = dict(title = 'Charge Ratio (from PLs or errorbars)'))

fig.write_image(pathfig+"Ratio_errorbars.pdf",engine='kaleido')
fig.show()